<a href="https://colab.research.google.com/github/MarcosVCS/Projeto-Final---Engenharia-de-Dados---SoulCode/blob/main/COEXP_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - instalar e importar

In [ ]:
import pandas as pd

In [ ]:
#Configurações de numero maximo de colunas em exibição
pd.set_option('display.max_columns',30)

In [ ]:
import numpy as np

google

In [ ]:
#instalar o gc file system
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importar a liberação da chave de acesso
import os
serviceAccount =  '/content/aula-bc26-flavia-75af138d1cdd.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
from google.cloud import storage

In [ ]:
from google.cloud.storage.blob import Blob 
import requests

In [ ]:
from google.oauth2 import service_account 
import os
import pandas_gbq 

from IPython.display import clear_output

spark

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=094f13b28ca49b1949f4be0c4b9f0325c1a40491555645b7deba995065476708
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('data-frames')
    .config('spark.ui.port','4050')
    .config('spark.jars', 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
    .getOrCreate()
)

In [ ]:
spark

#2 - Conector Goggle Storage

In [ ]:
# Upload do ARQUIVO DO BUCKET - Ana

# Chave de acesso à Bucket no Cloud Storage
serviceAccount = '/content/chave_bucket_JSON.json' 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

#indicar a bucket
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')
# bucket = client.get_bucket('comex-original-link')

In [ ]:
# # Para acessar o arquivo que está na bucket
# bucket.blob('bc26-datasets-comex/EXP_COMPLETA (NCM).csv')
# #para enviar
# blob = bucket.blob('preco-combustiveis-2013-2022')
# blob.upload_from_filename('/content/preco-combustiveis-2013-2022')

#Upload dos arquivos no Bucket do Cloud Storage

In [ ]:
#link das bases usadas da página do ministério da economia
exp_prod = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip'
imp_prod = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip'
exp_munc = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_COMPLETA_MUN.zip'
imp_munc = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/IMP_COMPLETA_MUN.zip'
#elas foram descompactadas, seu formato era csv, elas foram mudadas para parquet para um melhor desempenho e 
#enviadas para o bucket como orginial contendo os 10 anos de interrese.

#link da tabelas auxiliares, usadas para a tradução dos códigos.
cod_aux = 'https://balanca.economia.gov.br/balanca/bd/tabelas/TABELAS_AUXILIARES.xlsx'

NCM (Nomenclatura Comum do Mercosul) é um código de oito dígitos estabelecido pela legislação federal para identificar as mercadorias e facilitar o comércio internacional, obrigatoriamente na NF-e, NFA-e e NFC-e.

In [ ]:
#relação do código ncm com os outro códigos de classificação de protudos, junto com sua descrioção
ncm = 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv'

#retirar o conteudo do link
request_text = requests.get(ncm).content
#enviar o conteudo para o bucket, indicando o formato
bucket.blob('ncm').upload_from_string(request_text, 'text/csv')


base de serie historia mensais de preço de combustível

In [ ]:
# link da página da agencia nacional do petroleo, gas natural  e biocombustiveis
#séries históricas mensais
comb15 = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/shlp/mensal/mensal-municipios-2013-a-2015.xlsx'
comb18 = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/shlp/mensal/mensal-municipios-2016-a-2018.xlsx'
comb21 = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/shlp/mensal/mensal-municipios-2019-a-2021.xlsx'
comb22 = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/shlp/mensal/mensal-municipios-desde-jan2022.xlsx'

In [ ]:
#pegar o conteudo do link
request_text = requests.get(comb15).content
#enviar o conteudo para o bucket
bucket.blob('comex-original-link/municipio-2013-2015').upload_from_string(request_text, 'xlsx')

In [ ]:
#pegar o conteudo do link
request_text = requests.get(comb18).content
#enviar o conteudo para o bucket
bucket.blob('comex-original-link/municipio-2016-2018').upload_from_string(request_text, 'xlsx')

In [ ]:
#pegar o conteudo do link
request_text = requests.get(comb21).content
#enviar o conteudo para o bucket
bucket.blob('comex-original-link/municipio-2019-2021').upload_from_string(request_text, 'xlsx')

In [ ]:
#pegar o conteudo do link
request_text = requests.get(comb22).content
#enviar o conteudo para o bucket
bucket.blob('comex-original-link/municipio-apartir-2022').upload_from_string(request_text, 'xlsx')

In [ ]:
pathtest = 'gs://bc26-datasets-comex/comex-original-link/municipio-apartir-2022'
mun22 = pd.read_excel(pathtest, skiprows=16)
#pulas as primeiras 16 linha, pois a base só começa depois

In [ ]:
mun22.tail(20)
#abriu o arquivo correto!

In [ ]:
#link da série semanal em xlsb - a formatação da data a deforma
d = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/shlp/semanal/semanal-municipios-2013-a-2017.xlsb'
#pegar o conteudo do link
request_text = requests.get(d).content
#enviar o conteudo para o bucket
bucket.blob('comex-original-link/semana-municipio-2013-2015').upload_from_string(request_text, 'xlsx')

In [ ]:

di = pd.read_excel('', skiprows=16)

#tabela de numeros validados

código para ler sheet do excel

In [ ]:
#ler xlsx com sheet
df = pd.read_excel('file.xlsx', sheet_name=0)

In [ ]:
val = pd.read_csv('https://balanca.economia.gov.br/balanca/bd/repetro/exp_repetro.xlsx')

In [ ]:
exp = pd.read_csv('/content/EXP_TOTAIS_CONFERENCIA.csv', sep=';')
imp = pd.read_csv('/content/IMP_TOTAIS_CONFERENCIA.csv', sep=';')

In [ ]:
val
f_exp = exp.CO_ANO >2012
f_imp = exp.CO_ANO >2012

In [ ]:
df1 = exp.loc[f_exp]
df1

,ARQUIVO,CO_ANO,QT_ESTAT,KG_LIQUIDO,VL_FOB,NUMERO_LINHAS
16,EXP_2013.csv,2013,580193111059,557952370783,232544255606,1074709
17,EXP_2014.csv,2014,485576625760,576395634212,220923236838,1057794
18,EXP_2015.csv,2015,521472343947,636029338192,186782355063,1048388
19,EXP_2016.csv,2016,526840590057,644891914917,179526129214,1092692
20,EXP_2017.csv,2017,560480100308,691743297215,214988108353,1096623
21,EXP_2018.csv,2018,584234291249,705549984512,231889523399,1128981
22,EXP_2019.csv,2019,526083644677,678108051917,221126807647,1393174
23,EXP_2020.csv,2020,503843862451,697446598673,209180241655,1344556
24,EXP_2021.csv,2021,524626711400,700387165796,280814577460,1480295
25,EXP_2022.csv,2022,489301147355,679812308938,308362261446,1367084


In [ ]:
df1.sum()

ARQUIVO          EXP_2013.csvEXP_2014.csvEXP_2015.csvEXP_2016.c...
CO_ANO                                                       20175
QT_ESTAT                                             5302652428263
KG_LIQUIDO                                           6568316665155
VL_FOB                                               2286137496681
NUMERO_LINHAS                                             12084296
dtype: object

In [ ]:
df2 = imp.loc[f_imp]
df2

,ARQUIVO,CO_ANO,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,NUMERO_LINHAS
16,IMP_2013.csv,2013,237939643436,159565878109,241500886459,10585684919,195133145,1887697
17,IMP_2014.csv,2014,244079554147,165284900159,230823018796,9761601940,178751246,1891607
18,IMP_2015.csv,2015,207578980463,146470902082,173104259077,7217609201,138467852,1784459
19,IMP_2016.csv,2016,197580870135,138412341184,139321357653,5814053785,112463368,1690065
20,IMP_2017.csv,2017,224138358488,147377404392,158951444003,6780582952,123205174,1840472
21,IMP_2018.csv,2018,242600455025,151392468267,185321983502,7382864017,135366561,1893229
22,IMP_2019.csv,2019,238234076545,153404722395,185927967580,7099935633,134100825,1932755
23,IMP_2020.csv,2020,232969373637,143979868044,158786824879,7433078641,116311933,1784057
24,IMP_2021.csv,2021,276494571488,178410726540,219408049180,15125230521,157165534,1999946
25,IMP_2022.csv,2022,236358982722,158139876395,250831426836,17964935586,197726287,1856515


In [ ]:
df2.sum()

ARQUIVO          IMP_2013.csvIMP_2014.csvIMP_2015.csvIMP_2016.c...
CO_ANO                                                       20175
QT_ESTAT                                             2337974866086
KG_LIQUIDO                                           1542439087567
VL_FOB                                               1943977217965
VL_FRETE                                               95165577195
VL_SEGURO                                               1488691925
NUMERO_LINHAS                                             18560802
dtype: object

#Carregar base original do bucket para pré-analise

In [ ]:
# DOWNLOAD ARQUIVO DO BUCKET - MARCOS

# FAZER O ACESSO A BUCKET
serviceAccount = '/content/chave_bucket_JSON.json' # Chave de acesso à Bucket no Cloud Storage
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount
client = storage.Client()             

# CRIAR UMA VARIÁVEL PARA RECEBER O NOME DA BUCKET 
bucket = client.get_bucket('bc26-datasets-comex')

# Para acessar o arquivo que está na bucket
bucket.blob('bc26-datasets-comex/EXP_COMPLETA (NCM).csv')

# CRIAR UMA VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path = 'gs://bc26-datasets-comex/EXP_COMPLETA (NCM).csv'

In [ ]:
#ler a base origal do storage
df = pd.read_csv(path,sep=';')

In [ ]:
df

In [ ]:
#intalação para formato diferente
pip install pyxlsb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
path2 = 'gs://bc26-datasets-comex/semanal-municipios-2013-a-2017.xlsb'
gas = pd.read_excel(path2, skiprows=14)

In [ ]:
gas

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,41273,41279,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,ETANOL HIDRATADO,80,R$/l,2.273,0.022864,2.190,2.299,0.405,0.010,1.868,0.016954,1.8429,1.8873,0.009
1,41273,41279,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,2.134,0.043359,2.090,2.190,0.268,0.020,1.866,0.020789,1.8512,1.8806,0.011
2,41273,41279,CENTRO OESTE,GOIAS,ANAPOLIS,ETANOL HIDRATADO,16,R$/l,1.865,0.054575,1.690,1.899,0.36,0.029,1.505,0.044552,1.4599,1.5527,0.03
3,41273,41279,CENTRO OESTE,GOIAS,APARECIDA DE GOIANIA,ETANOL HIDRATADO,13,R$/l,1.771,0.022532,1.730,1.790,0.206,0.013,1.565,0.072579,1.5199,1.6737,0.046
4,41273,41279,CENTRO OESTE,GOIAS,CALDAS NOVAS,ETANOL HIDRATADO,12,R$/l,1.918,0.105396,1.650,1.999,0.376,0.055,1.542,0.046058,1.4734,1.5693,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657867,43093,43099,SUDESTE,SAO PAULO,TABOAO DA SERRA,GNV,1,R$/m3,2.290,0.000000,2.290,2.290,-,0.000,-,-,-,-,-
657868,43093,43099,CENTRO OESTE,MATO GROSSO DO SUL,TRES LAGOAS,GNV,1,R$/m3,2.399,0.000000,2.399,2.399,-,0.000,-,-,-,-,-
657869,43093,43099,SUDESTE,RIO DE JANEIRO,TRES RIOS,GNV,3,R$/m3,2.999,0.000000,2.999,2.999,1.061,0.000,1.938,0,1.938,1.938,0
657870,43093,43099,SUDESTE,ESPIRITO SANTO,VITORIA,GNV,1,R$/m3,2.399,0.000000,2.399,2.399,-,0.000,-,-,-,-,-


In [ ]:
gas.dtypes

DATA INICIAL                       int64
DATA FINAL                         int64
REGIÃO                            object
ESTADO                            object
MUNICÍPIO                         object
PRODUTO                           object
NÚMERO DE POSTOS PESQUISADOS       int64
UNIDADE DE MEDIDA                 object
PREÇO MÉDIO REVENDA              float64
DESVIO PADRÃO REVENDA            float64
PREÇO MÍNIMO REVENDA             float64
PREÇO MÁXIMO REVENDA             float64
MARGEM MÉDIA REVENDA              object
COEF DE VARIAÇÃO REVENDA         float64
PREÇO MÉDIO DISTRIBUIÇÃO          object
DESVIO PADRÃO DISTRIBUIÇÃO        object
PREÇO MÍNIMO DISTRIBUIÇÃO         object
PREÇO MÁXIMO DISTRIBUIÇÃO         object
COEF DE VARIAÇÃO DISTRIBUIÇÃO     object
dtype: object

In [ ]:
#Renomeando colunas
df2.rename(columns={'CO_ANO':'ano',
                   'CO_MES':'mes',
                   'CO_NCM':'codigo_mercadoria',
                   'CO_UNID':'codigo_medida',
                    'CO_PAIS':'pais',
                    'SG_UF_NCM':'estado',
                    'CO_VIA':'cod_meio_transporte',
                    'CO_URF':'cod_aduaneiro',
                    'QT_ESTAT':'qnt_estatistica',
                    'KG_LIQUIDO':'peso_liquido',
                    'VL_FOB':'preco'
                    },inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
f_mes = df.CO_MES == 1
f_ano = df.CO_ANO == 2020
dfjan20 = df.loc[f_mes & f_ano]

In [ ]:
dfjan20
d_est = dfjan20.CO_UNID == 10

In [ ]:
dfjan = dfjan20.loc[d_est]

In [ ]:
dfjan

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
21557974,2020,1,39211900,10,244,SP,4,817600,1,1,2
21557998,2020,1,20082090,10,850,GO,1,817800,1440,1440,4158
21558016,2020,1,31052000,10,586,PR,7,147700,2347000,2347000,775700
21558018,2020,1,84139190,10,845,SP,7,1017701,437,437,34669
21558029,2020,1,7032010,10,628,RJ,1,717600,10,10,40
...,...,...,...,...,...,...,...,...,...,...,...
22902462,2020,1,19021900,10,337,AM,0,260152,40,40,72
22902494,2020,1,7108000,10,137,PR,1,917800,5,5,10
22902505,2020,1,85030090,10,676,SC,1,927700,12,12,24
22902507,2020,1,8039000,10,267,SP,12,817800,35,35,26


In [ ]:
dfjan.CO_NCM.is_unique
dfjan.SG_UF_NCM.is_unique
f7032010 = dfjan.CO_NCM == 7032010

In [ ]:
df7032010 =dfjan.loc[f7032010]
df7032010

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
21558029,2020,1,7032010,10,628,RJ,1,717600,10,10,40
21622159,2020,1,7032010,10,580,RS,1,1017700,10,10,55
21748896,2020,1,7032010,10,434,RS,1,1017700,21,21,166
21838181,2020,1,7032010,10,23,RJ,1,717600,5,5,7
21884434,2020,1,7032010,10,538,RS,1,1017700,15,15,108
21921986,2020,1,7032010,10,351,PR,1,917800,25,25,85
21925272,2020,1,7032010,10,83,RS,1,1017700,10,10,60
21948369,2020,1,7032010,10,741,RS,1,1017700,7,7,56
22203190,2020,1,7032010,10,476,RS,1,1017700,38,38,264
22294967,2020,1,7032010,10,361,RJ,1,717800,10,10,42


spark

In [ ]:
dfsjan20 = spark.createDataFrame(dfjan20)

In [ ]:
dfsjan20.show()

#tabelas auxiliares xlsx com sheets

##pegando todos os sheet das tabelas auxiliares

In [ ]:
#abrir todas as sheet
# Para acessar o arquivo que está na bucket
bucket.blob('bc26-datasets-comex/TABELAS_AUXILIARES.xlsx')

# CRIAR UMA VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path = 'gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx'

In [ ]:
#laço de repetição para adicionar posições na lista.
cod = []
for i in range(16):
  cod.append(str('df')+str(i))
  print(cod[i])

df0
df1
df2
df3
df4
df5
df6
df7
df8
df9
df10
df11
df12
df13
df14
df15


SH - sistema harminizado
CUCI - Classificação Uniforme para Comercio Internacional
CGCE - Classificação por Grandes Categorias Econômicas
ISIC - International Standard Industruial Classification
SIIT - Setores Industriais por Intensidade Tecnológica
PPE - Pauta de Protudos Exportados (categoria da SECEX)
PPI - Pauta de Portudos Importados (categoria da SECEX)

In [ ]:
cod = []

#abrir a sheets
for i in range(16):
  cod.append('i')
  cod[i] = pd.read_excel(path, sheet_name=(i+1))
#Carregar as 16 sheet da tabel auxiliar

##analisando os sheet

In [ ]:
#relação entre NCM e SH
cod[0]
#tem nomes em portugues, espenhol e ingles do SH4 e SH6
#o SH6 é o NCM sem os 2 últimos dígitos
#o SH$ é o NCM sem os 4 últimos dígitos
#o SH$ é o NCM sem os 6 últimos dígitos
#o sec é categoria

,CO_NCM,NO_NCM_POR,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,1011010,Cavalos reprodutores de raça pura,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
2,1011100,"Cavalos reprodutores, de raça pura",10111,"Cavalos reprodutores, de raça pura",Caballos reproductores de raza pura,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
3,1011900,"Outros cavalos, vivos",10119,"Outros cavalos, vivos","Demás caballos, vivos",Other live horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
4,1012000,Asininos e muares vivos,10120,Asininos e muares vivos,"Asnos, mulos y burdéganos, vivos","Asses, mules and hinnies (live)",101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71","Piedras, en bruto o no, del capítulo 71","Stones, raw or worked, from Chapter 71",9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13594,99997104,Outros artigos do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71","Piedras, en bruto o no, del capítulo 71","Stones, raw or worked, from Chapter 71",9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,999999,Qualquer outra mercadoria sem cobertura cambia...,Todos otros bienes sin cobertura de cambios y...,Any other goods without exchange cover; reexport,9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13596,99999951,Devolução antes da DI (expor

In [ ]:
#Relação NCM com CUCI, Grupo/divisão CUCI e segmento
cod[1]

,CO_NCM,NO_NCM_POR,CO_CUCI_ITEM,NO_CUCI_ITEM,CO_CUCI_SUB,NO_CUCI_SUB,CO_CUCI_GRUPO,NO_CUCI_GRUPO,CO_CUCI_DIVISAO,NO_CUCI_DIVISAO,CO_CUCI_SEC,NO_CUCI_SEC
0,1011010,Cavalos reprodutores de raça pura,0015,"Cavalos, asnos, mulas e burros vivos",15,"Cavalos, asnos, mulas e burros vivos",001,"Animais vivos, não incluído pescados ou crustá...",0,Animais vivos não incluídos no capítulo 03,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",0015,"Cavalos, asnos, mulas e burros vivos",15,"Cavalos, asnos, mulas e burros vivos",001,"Animais vivos, não incluído pescados ou crustá...",0,Animais vivos não incluídos no capítulo 03,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS
2,1011100,"Cavalos reprodutores, de raça pura",0015,"Cavalos, asnos, mulas e burros vivos",15,"Cavalos, asnos, mulas e burros vivos",001,"Animais vivos, não incluído pescados ou crustá...",0,Animais vivos não incluídos no capítulo 03,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS
3,1011900,"Outros cavalos, vivos",0015,"Cavalos, asnos, mulas e burros vivos",15,"Cavalos, asnos, mulas e burros vivos",001,"Animais vivos, não incluído pescados ou crustá...",0,Animais vivos não incluídos no capítulo 03,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS
4,1012000,Asininos e muares vivos,0015,"Cavalos, asnos, mulas e burros vivos",15,"Cavalos, asnos, mulas e burros vivos",001,"Animais vivos, não incluído pescados ou crustá...",0,Animais vivos não incluídos no capítulo 03,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS
...,...,...,...,...,...,...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,9310,Operações especiais e commodities não classifi...,9310,Operações especiais e commodities não classifi...,931,Operações especiais e commodities não classifi...,93,Operações especiais e commodities não classifi...,9,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13594,99997104,Outros artigos do capitulo 71 da NCM,9310,Operações especiais e commodities não classifi...,9310,Operações especiais e commodities não classifi...,931,Operações especiais e commodities não classifi...,93,Operações especiais e commodities não classifi...,9,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,9310,Operações especiais e commodities não classifi...,9310,Operações especiais e commodities não classifi...,931,Operações especiais e commodities não classifi...,93,Operações especiais e commodities não classifi...,9,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13596,99999951,Devolução antes da DI (exportação definitiva),9310,Operações especiais e commodities não classifi...,9310,Operações especiais e commodities não classifi...,931,Operações especiais e commodities não classifi...,93,Operações especiais e commodities não classifi...,9,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...


In [ ]:
#relação NCM com CGCE n3/n2 e n1 (tipo de bens)
cod[2]

,CO_NCM,NO_NCM_POR,CO_CGCE_N3,NO_CGCE_N3,NO_CGCE_N3_ING,NO_CGCE_N3_ESP,CO_CGCE_N2,NO_CGCE_N2,NO_CGCE_N2_ING,NO_CGCE_N2_ESP,CO_CGCE_N1,NO_CGCE_N1,NO_CGCE_N1_ING,NO_CGCE_N1_ESP
0,1011010,Cavalos reprodutores de raça pura,110,Bens de capital (exceto equipamentos de transp...,"Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",11,"Bens de capital, exceto equipamentos de transp...","Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",1,BENS DE CAPITAL (BK),CAPITAL GOODS (BK),BIENES DE CAPITAL (BK)
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",110,Bens de capital (exceto equipamentos de transp...,"Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",11,"Bens de capital, exceto equipamentos de transp...","Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",1,BENS DE CAPITAL (BK),CAPITAL GOODS (BK),BIENES DE CAPITAL (BK)
2,1011100,"Cavalos reprodutores, de raça pura",110,Bens de capital (exceto equipamentos de transp...,"Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",11,"Bens de capital, exceto equipamentos de transp...","Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",1,BENS DE CAPITAL (BK),CAPITAL GOODS (BK),BIENES DE CAPITAL (BK)
3,1011900,"Outros cavalos, vivos",210,"Alimentos e bebidas básicos, destinados princi...","Basic food and beverages, mainly for industry","Alimentos y bebidas básicos, principalmente pa...",21,"Alimentos e bebidas básicos, destinados princi...","Basic food and beverages, mainly for industry","Alimentos y bebidas básicos, principalmente pa...",2,BENS INTERMEDIÁRIOS (BI),INTERMEDIATE GOODS (BI),BIENES INTERMEDIOS (BI)
4,1012000,Asininos e muares vivos,110,Bens de capital (exceto equipamentos de transp...,"Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",11,"Bens de capital, exceto equipamentos de transp...","Capital goods, except industrial transport equ...","Bienes de capital, excepto equipo de transport...",1,BENS DE CAPITAL (BK),CAPITAL GOODS (BK),BIENES DE CAPITAL (BK)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,311,Bens de consumo duráveis – exceto equipamentos...,Durable consumer goods - except transport equi...,Bienes de consumo durables - excepto equipo de...,31,Bens de consumo duráveis,Durable consumer goods,Bienes de consumo durables,3,BENS DE CONSUMO (BC),CONSUMER GOODS (BC),BIENES DE CONSUMO (BC)
13594,99997104,Outros artigos do capitulo 71 da NCM,322,Bens de consumo não duráveis,Non-durable goods,Bienes de consumo no durables,32,Bens de consumo semiduráveis e não duráveis,Consumer goods semi-durable and non-durable,Bienes de consumo semidurables y no durables,3,BENS DE CONSUMO (BC),CONSUMER GOODS (BC),BIENES DE CONSUMO (BC)
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,999,Bens não especificados anteriormente,Goods not otherwise specified,Bienes no especificados en otra categoría,99,Bens não especificados anteriormente,Goods not otherwise specified,Bienes no especificados en otra categoría,9,BENS NÃO ESPECIFICADOS ANTERIORMENTE,GOODS NOT SPECIFIED PREVIOUSLY,PROPIEDAD NO EXPRESADAS ANTERIORMENTE
13596,99999951,Devolução antes da DI (exportação definitiva),999,Bens não especificados anteriormente,Goods not otherwise specified,Bienes no especificados en otra categoría,99,Bens não especificados anteriormente,Goods not otherwise specified,Bienes no especificados en otra categoría,9,BENS NÃO ESPECIFICADOS ANTERIORMENTE,GOODS NOT SPECIFIED PREVIOUSLY,PROPIEDAD NO EXPRESADAS ANTERIORMENTE


In [ ]:
#relação NCM com ISIC classe, grupo, divisão, seção (aréa de atuação)
cod[3]

,CO_NCM,NO_NCM_POR,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_CLASSE_ING,NO_ISIC_CLASSE_ESP,CO_ISIC_GRUPO,NO_ISIC_GRUPO,NO_ISIC_GRUPO_ING,NO_ISIC_GRUPO_ESP,CO_ISIC_DIVISAO,NO_ISIC_DIVISAO,NO_ISIC_DIVISAO_ING,NO_ISIC_DIVISAO_ESP,CO_ISIC_SECAO,NO_ISIC_SECAO,NO_ISIC_SECAO_ING,NO_ISIC_SECAO_ESP
0,1011010,Cavalos reprodutores de raça pura,142,Criação de cavalos e outros equinos,Raising of horses and other equines,Criação de cavalos e outros equinos,14,Produção animal,Animal production,Produção animal,1,"Produção vegetal, animal e caça","Crop and animal production, hunting and relate...","Produção vegetal, animal e caça",A,Agropecuária,"Agriculture, forestry and fishing",Agropecuária
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",142,Criação de cavalos e outros equinos,Raising of horses and other equines,Criação de cavalos e outros equinos,14,Produção animal,Animal production,Produção animal,1,"Produção vegetal, animal e caça","Crop and animal production, hunting and relate...","Produção vegetal, animal e caça",A,Agropecuária,"Agriculture, forestry and fishing",Agropecuária
2,1011100,"Cavalos reprodutores, de raça pura",142,Criação de cavalos e outros equinos,Raising of horses and other equines,Criação de cavalos e outros equinos,14,Produção animal,Animal production,Produção animal,1,"Produção vegetal, animal e caça","Crop and animal production, hunting and relate...","Produção vegetal, animal e caça",A,Agropecuária,"Agriculture, forestry and fishing",Agropecuária
3,1011900,"Outros cavalos, vivos",142,Criação de cavalos e outros equinos,Raising of horses and other equines,Criação de cavalos e outros equinos,14,Produção animal,Animal production,Produção animal,1,"Produção vegetal, animal e caça","Crop and animal production, hunting and relate...","Produção vegetal, animal e caça",A,Agropecuária,"Agriculture, forestry and fishing",Agropecuária
4,1012000,Asininos e muares vivos,142,Criação de cavalos e outros equinos,Raising of horses and other equines,Criação de cavalos e outros equinos,14,Produção animal,Animal production,Produção animal,1,"Produção vegetal, animal e caça","Crop and animal production, hunting and relate...","Produção vegetal, animal e caça",A,Agropecuária,"Agriculture, forestry and fishing",Agropecuária
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,8999,Não classificado,Not Classified,Não classificado,899,Não classificado,Not Classified,Não classificado,89,Não classificado,Not Classified,Não classificado,D,Outros Produtos,Other products,Outros Produtos
13594,99997104,Outros artigos do capitulo 71 da NCM,8999,Não classificado,Not Classified,Não classificado,899,Não classificado,Not Classified,Não classificado,89,Não classificado,Not Classified,Não classificado,D,Outros Produtos,Other products,Outros Produtos
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,8999,Não classificado,Not Classified,Não classificado,899,Não classificado,Not Classified,Não classificado,89,Não classificado,Not Classified,Não classificado,D,Outros Produtos,Other products,Outros Produtos
13596,99999951,Devolução antes da DI (exportação definitiva),8999,Não classificado,Not Classified,Não classificado,899,Não classificado,Not Classified,Não classificado,89,Não classificado,Not Classified,Não classificado,D,Outros Produtos,Other products,Outros Produtos


In [ ]:
#relação NCM o com o ISIC seção e grupo
cod[15]

,CO_NCM,NO_NCM_POR,CO_ISIC_SECAO,NO_ISIC_SECAO,CO_CUCI_GRUPO,NO_CUCI_GRUPO
0,1011010,Cavalos reprodutores de raça pura,A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustá..."
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustá..."
2,1011100,"Cavalos reprodutores, de raça pura",A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustá..."
3,1011900,"Outros cavalos, vivos",A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustá..."
4,1012000,Asininos e muares vivos,A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustá..."
...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,D,Outros Produtos,931,Operações especiais e commodities não classifi...
13594,99997104,Outros artigos do capitulo 71 da NCM,D,Outros Produtos,931,Operações especiais e commodities não classifi...
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,D,Outros Produtos,931,Operações especiais e commodities não classifi...
13596,99999951,Devolução antes da DI (exportação definitiva),D,Outros Produtos,931,Operações especiais e commodities não classifi...


In [ ]:
#relaciona o NCM com o SIIT
cod[4]
#4 categorias de SIIT pelo uso de tecnologia na industria (Premissa: produtos com valor de agregação alto ou baixo)
pd.unique(cod[4].CO_SIIT)
pd.unique(cod[4].NO_SIIT)

array(['PRODUTOS N.C.I.T',
       'PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIXA TECNOLOGIA',
       'PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDIA-ALTA TECNOLOGIA',
       'PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDIA-BAIXA TECNOLOGIA',
       'PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE ALTA TECNOLOGIA'],
      dtype=object)

In [ ]:
cod[5].tail(30)
#indica qual a unidade de medida usada com esse NCM

,CO_NCM,NO_NCM_POR,CO_UNID,NO_UNID,SG_UNID
13568,97031000,Produções originais de arte estatuária ou de e...,11,NUMERO (UNIDADE),UNID.
13569,97039000,Outras produções originais de arte estatuária ...,11,NUMERO (UNIDADE),UNID.
13570,97040000,"Selos postais, selos fiscais, marcas postais, ...",10,QUILOGRAMA LIQUIDO,KGL
13571,97050000,"Coleções e espécimes para coleções, de zoologi...",10,QUILOGRAMA LIQUIDO,KGL
13572,97051000,Coleções e peças de coleção que apresentem um ...,10,QUILOGRAMA LIQUIDO,KGL
13573,97052100,Espécimes humanos e suas partes,10,QUILOGRAMA LIQUIDO,KGL
13574,97052900,Outras coleções e peças de coleção que apresen...,10,QUILOGRAMA LIQUIDO,KGL
13575,97053900,Outras coleções e peças de coleção que aprese...,10,QUILOGRAMA LIQUIDO,KGL
13576,97060000,Antiguidades com mais de 100 anos,10,QUILOGRAMA LIQUIDO,KGL
13577,97061000,Antiguidades com mais de 250 anos,10,QUILOGRAMA LIQUIDO,KGL


In [ ]:
cod[6]
#NCM com fator de agregação
pd.unique(cod[6].NO_FAT_AGREG)
#tem 6 categorias
pd.unique(cod[6].NO_FAT_AGREG_GP)
#tem 3 categorias

array(['PRODUTOS BASICOS', 'PRODUTOS INDUSTRIALIZADOS',
       'OPERACOES ESPECIAIS'], dtype=object)

In [ ]:
cod[7]
#o PPI atribuido ao NCM, código de produto
pd.unique(cod[7].NO_PPI)
cod[8]
#o PPE de cada NCM 

,CO_NCM,NO_NCM_POR,CO_PPE,NO_PPE,NO_PPE_MIN,NO_PPE_ING
0,1011010,Cavalos reprodutores de raça pura,1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS
2,1011100,"Cavalos reprodutores, de raça pura",1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS
3,1011900,"Outros cavalos, vivos",1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS
4,1012000,Asininos e muares vivos,1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS
...,...,...,...,...,...,...
13317,99997103,Joalheria de ouro do capitulo 71 da NCM,4020,TRANSACOES ESPECIAIS - PEDRAS EM BRUTO DO CAP....,Transações especiais - pedras em bruto do cap....,SPECIAL TRANSACTIONS - CRUDE STONES OF CAP.71 ...
13318,99997104,Outros artigos do capitulo 71 da NCM,4020,TRANSACOES ESPECIAIS - PEDRAS EM BRUTO DO CAP....,Transações especiais - pedras em bruto do cap....,SPECIAL TRANSACTIONS - CRUDE STONES OF CAP.71 ...
13319,99999900,Qualquer outra mercadoria sem cobertura cambial,4010,TRANSACOES ESPECIAIS - DEMAIS,Transações especiais - demais,SPECIAL TRANSACTIONS - OTHER
13320,99999951,Devolução antes da DI (exportação definitiva),6010,REEXPORTACAO,Reexportação,REEXPORT


In [ ]:
cod[9]
#nome e sigla dos países para CO_pais
cod[10]
#indica o bloco do pais, pelo CO_Pais

,CO_PAIS,CO_BLOCO,NO_BLOCO,NO_BLOCO_ING,NO_BLOCO_ESP
0,788,51,África,Africa (minus MIDDLE EAST),África (menos ORIENTE MEDIO)
1,525,51,África,Africa (minus MIDDLE EAST),África (menos ORIENTE MEDIO)
2,229,51,África,Africa (minus MIDDLE EAST),África (menos ORIENTE MEDIO)
3,281,51,África,Africa (minus MIDDLE EAST),África (menos ORIENTE MEDIO)
4,438,51,África,Africa (minus MIDDLE EAST),África (menos ORIENTE MEDIO)
...,...,...,...,...,...
317,15,22,União Europeia - UE,European Union (EU),Uniao Europea (UE)
318,23,22,União Europeia - UE,European Union (EU),Uniao Europea (UE)
319,393,22,União Europeia - UE,European Union (EU),Uniao Europea (UE)
320,791,22,União Europeia - UE,European Union (EU),Uniao Europea (UE)


In [ ]:
cod[11]
#pela CO-UF diz a sigla, nome do estado e país
cod[12]
# pelo CO-MUN_GEO indica nome do município e estado

,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,1501758,BREJO GRANDE DO ARAGUAIA,Brejo Grande do Araguaia,PA
1,1501782,BREU BRANCO,Breu Branco,PA
2,1501808,BREVES,Breves,PA
3,1501907,BUJARU,Bujaru,PA
4,1501956,CACHOEIRA DO PIRIA,Cachoeira do Piriá,PA
...,...,...,...,...
5565,4322350,UNIAO DA SERRA,União da Serra,RS
5566,4322376,UNISTALDA,Unistalda,RS
5567,4322400,URUGUAIANA,Uruguaiana,RS
5568,4322509,VACARIA,Vacaria,RS


In [ ]:
cod[13]
#pelo CO_VIA diz o meio de transporte
#tem não declareda e desconhecida

,CO_VIA,NO_VIA
0,0,VIA NAO DECLARADA
1,1,MARITIMA
2,2,FLUVIAL
3,3,LACUSTRE
4,4,AEREA
5,5,POSTAL
6,6,FERROVIARIA
7,7,RODOVIARIA
8,8,CONDUTO/REDE DE TRANSMISSAO
9,9,MEIOS PROPRIOS


In [ ]:
cod[14]
#pelo CO-URF indica de onde saiu, contem não informados
pd.unique(cod[14].NO_URF)
pontos_saida = pd.unique(cod[14].NO_URF)
len(pontos_saida)

277

##montar dataframe com as colunas de classificação

In [ ]:
#verirficar se as colunas são únicas para usar como atributo-chave
cod[0].CO_NCM.is_unique
cod[0].shape
cod[0].CO_SH6.is_unique
cod[0].dtypes

CO_NCM            int64
NO_NCM_POR       object
CO_SH6            int64
NO_SH6_POR       object
NO_SH6_ESP       object
NO_SH6_ING       object
CO_SH4            int64
NO_SH4_POR       object
NO_SH4_ESP       object
NO_SH4_ING       object
CO_SH2            int64
NO_SH2_POR       object
NO_SH2_ESP       object
NO_SH2_ING       object
CO_NCM_SECROM    object
NO_SEC_POR       object
NO_SEC_ESP       object
NO_SEC_ING       object
dtype: object

In [ ]:
#entidade chave -NCM cod, NCM descrição - menor nível de classificação

#acrescetar o sh6 correspondente sh6 - codigo, sh2-capitulo, sh4- posição no capitulo.
#colocar o sh6 cod e nome, sh2 e sh 4 só nome
#co_ncm_secron tem 22 valores - é a seção do NCM
#Esses códigos vão do menor envolvimento humano (animais vivos) ao máximo (obras de arte)

In [ ]:
#selecionar as colunas criando um novo dataframe
df_sh = cod[0].loc[:,['CO_NCM','NO_NCM_POR','CO_SH6','NO_SH6_POR','NO_SH4_POR','NO_SH2_POR','NO_SEC_POR']]
df_sh.shape

(13598, 7)

In [ ]:
#verifciar se a coluna é única para usa-la como atributi chave
cod[1].CO_NCM.is_unique
cod[1].shape
cod[1].dtypes

CO_NCM              int64
NO_NCM_POR         object
CO_CUCI_ITEM       object
NO_CUCI_ITEM       object
CO_CUCI_SUB         int64
NO_CUCI_SUB        object
CO_CUCI_GRUPO      object
NO_CUCI_GRUPO      object
CO_CUCI_DIVISAO     int64
NO_CUCI_DIVISAO    object
CO_CUCI_SEC         int64
NO_CUCI_SEC        object
dtype: object

In [ ]:
#classificação internacional intermediária
df_cuci = cod[1].loc[:,['CO_NCM','CO_CUCI_ITEM','NO_CUCI_ITEM','NO_CUCI_SUB','NO_CUCI_GRUPO','NO_CUCI_DIVISAO','NO_CUCI_SEC']]
df_cuci.shape

(13598, 7)

In [ ]:
#verificar novamente
cod[2].CO_NCM.is_unique
cod[2].shape
cod[2].dtypes

CO_NCM             int64
NO_NCM_POR        object
CO_CGCE_N3         int64
NO_CGCE_N3        object
NO_CGCE_N3_ING    object
NO_CGCE_N3_ESP    object
CO_CGCE_N2         int64
NO_CGCE_N2        object
NO_CGCE_N2_ING    object
NO_CGCE_N2_ESP    object
CO_CGCE_N1         int64
NO_CGCE_N1        object
NO_CGCE_N1_ING    object
NO_CGCE_N1_ESP    object
dtype: object

In [ ]:
#class de grandes categorias economicas
df_cgce = cod[2].loc[:,['CO_NCM','CO_CGCE_N3','NO_CGCE_N3','NO_CGCE_N2','NO_CGCE_N1']]
df_cgce.shape

(13598, 5)

In [ ]:
#ver as respostas ditintas da coluna
pd.unique(df_cgce.CO_CGCE_N3)

array([110, 210, 324, 220, 322, 323, 230, 240, 421, 410, 420, 321, 270,
       280, 311, 999, 120, 312, 313])

In [ ]:
cod[3].CO_NCM.is_unique
#ver as informações dessa expecífica tabela auxiliar
cod[3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13598 entries, 0 to 13597
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   CO_NCM               13598 non-null  int64 
 1   NO_NCM_POR           13598 non-null  object
 2   CO_ISIC_CLASSE       13598 non-null  int64 
 3   NO_ISIC_CLASSE       13598 non-null  object
 4   NO_ISIC_CLASSE_ING   13598 non-null  object
 5   NO_ISIC_CLASSE_ESP   13598 non-null  object
 6   CO_ISIC_GRUPO        13598 non-null  int64 
 7   NO_ISIC_GRUPO        13598 non-null  object
 8   NO_ISIC_GRUPO_ING    13598 non-null  object
 9   NO_ISIC_GRUPO_ESP    13598 non-null  object
 10  CO_ISIC_DIVISAO      13598 non-null  int64 
 11  NO_ISIC_DIVISAO      13598 non-null  object
 12  NO_ISIC_DIVISAO_ING  13598 non-null  object
 13  NO_ISIC_DIVISAO_ESP  13598 non-null  object
 14  CO_ISIC_SECAO        13598 non-null  object
 15  NO_ISIC_SECAO        13598 non-null  object
 16  NO_I

In [ ]:
#isic - class internacional de atividades economias, sustitui o fator agregado, com 4 grandes grupos.
df_isic = cod[3].loc[:,['CO_NCM','CO_ISIC_CLASSE','NO_ISIC_CLASSE','NO_ISIC_GRUPO','NO_ISIC_DIVISAO','NO_ISIC_SECAO']]
df_isic.shape

(13598, 6)

In [ ]:
df_cuci.tail()

,CO_NCM,CO_CUCI_ITEM,NO_CUCI_ITEM,NO_CUCI_SUB,NO_CUCI_GRUPO,NO_CUCI_DIVISAO,NO_CUCI_SEC
13593,99997103,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13594,99997104,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13595,99999900,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13596,99999951,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...
13597,99999960,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...


In [ ]:
df_isic.head()

,CO_NCM,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_GRUPO,NO_ISIC_DIVISAO,NO_ISIC_SECAO
0,1011010,142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária
1,1011090,142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária
2,1011100,142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária
3,1011900,142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária
4,1012000,142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária


In [ ]:
#correlaçao isic seção com cuci grupo.
cod[15].CO_NCM.is_unique
cod[15].shape
cod[15].dtypes

CO_NCM            int64
NO_NCM_POR       object
CO_ISIC_SECAO    object
NO_ISIC_SECAO    object
CO_CUCI_GRUPO    object
NO_CUCI_GRUPO    object
dtype: object

In [ ]:
cod[4].CO_NCM.is_unique
cod[4].shape
cod[4].dtypes

CO_NCM         int64
NO_NCM_POR    object
CO_SIIT        int64
NO_SIIT       object
dtype: object

In [ ]:
#siit - setores industriais por intensidade tecnologia
df_siit = cod[4].loc[:,['CO_NCM','NO_SIIT']]
df_siit.shape

(13322, 2)

In [ ]:
cod[5].CO_NCM.is_unique
cod[5].shape
cod[5].dtypes
#as unidades foram traduzidas nas bases
len(pd.unique(cod[5].CO_UNID))
pd.unique(cod[5].NO_UNID)

array(['NUMERO (UNIDADE)', 'QUILOGRAMA LIQUIDO', 'DUZIA', 'GRAMA LIQUIDO',
       'TONELADA METRICA LIQUIDA', 'LITRO', 'MILHEIRO', 'METRO CUBICO',
       'MIL QUILOWATT HORA', 'QUILATE', 'METRO QUADRADO', 'METRO',
       'PARES'], dtype=object)

In [ ]:
#o fator de agregação está sendo descontinuado pelo sercretria, usar o isic no lugar, 
cod[6].CO_NCM.is_unique
cod[6].shape
cod[6].dtypes
pd.unique(cod[6].NO_FAT_AGREG)

array(['PRODUTOS BASICOS', 'PRODUTOS MANUFATURADOS',
       'PRODUTOS SEMIMANUFATURADOS', 'TRANSACOES ESPECIAIS',
       'CONSUMO DE BORDO', 'REEXPORTACAO'], dtype=object)

In [ ]:
cod[7].CO_NCM.is_unique
cod[8].CO_NCM.is_unique
cod[7].shape
cod[7].dtypes
cod[8].dtypes
pd.unique(cod[7].NO_PPI)
#ver quantas respostas unicas tem o atributo
len(pd.unique(cod[7].NO_PPI))
lista_imp = pd.unique(cod[7].NO_PPI).sort()
pd.unique(cod[8].NO_PPE)
len(pd.unique(cod[8].NO_PPE))
lista_exp = pd.unique(cod[8].NO_PPE).sort()

In [ ]:
#lista de paute de produtos para impotação é igual a de exportação
lista_imp == lista_exp
#tema mesma finalidade que a CGCE.

True

##montar df de paises e blocos

pais - nomes

In [ ]:
# cod[9] e cod[10] juntar o cod do país com nomo e bloco
cod[9].dtypes
cod[9].head()

In [ ]:
#ver colunas 
cod[9].dtypes 
#colunas a serem selecionas CO_PAIS NO_PAIS

# montar o dataframe com o nome e código de pais
dfpais = cod[9].loc[:,['CO_PAIS','NO_PAIS']]
dfpais.shape


(281, 2)

In [ ]:
#renoar as colunas
dfpais.rename(columns={ 'CO_PAIS': 'cod_pais', 'NO_PAIS':'pais'}, inplace=True)
dfpais

,cod_pais,pais
0,0,Não Definido
1,13,Afeganistão
2,15,"Aland, Ilhas"
3,17,Albânia
4,20,"Alboran-Perejil, Ilhas"
...,...,...
276,994,A Designar
277,995,Bancos Centrais
278,997,Organizações Internacionais
279,998,Sem informação


In [ ]:
#verificar se contem registros nulos
dfpais.isna().sum()

cod_pais    0
pais        0
dtype: int64

paises - blocos

In [ ]:
#só preciso do nome do país
cod[10].dtypes
cod[10].head()
#só preciso do nome do bloco
pd.unique(cod[10].NO_BLOCO)

array(['África', 'América Central e Caribe', 'América do Norte',
       'América do Sul', 'Ásia (Exclusive Oriente Médio)',
       'Associação de Nações do Sudeste Asiático - ASEAN', 'Europa',
       'Mercado Comum do Sul - Mercosul', 'Oceania', 'Oriente Médio',
       'União Europeia - UE'], dtype=object)

In [ ]:
cod[9].CO_PAIS.is_unique
cod[9].shape
#são 281 paises
cod[10].CO_PAIS.is_unique
#paises repetitos, separar os blocos geagraficos e economicos

False

In [ ]:
#voltei nesse comando
cod[9] = pd.read_excel(path, sheet_name=(10))
cod[10] = pd.read_excel(path, sheet_name=(11))

In [ ]:
#filtro para blocos geograficos
filtro_bl = cod[10].NO_BLOCO == ('África' or 'América Central e Caribe' or 'América do Norte' or
       'América do Sul' or 'Ásia (Exclusive Oriente Médio)' or 'Europa' or 'Oceania' or 'Oriente Médio')
dfbloc = cod[10].loc[filtro_bl]
dfbloc.CO_PAIS.is_unique

True

In [ ]:
#filtro de blocos economicos
filtro_ec = cod[10].NO_BLOCO == ('Associação de Nações do Sudeste Asiático - ASEAN' or 'Mercado Comum do Sul - Mercosul' or 'União Europeia - UE')
dfeco = cod[10].loc[filtro_ec]
dfeco.CO_PAIS.is_unique

True

In [ ]:
dfeco.dtypes #CO_PAIS NO_BLOCO
#cria o df de blocas economico
dfeco = dfeco.loc[:,['CO_PAIS', 'NO_BLOCO']]
#renomear as colunas
dfeco.rename(columns={'CO_PAIS': 'cod_pais','NO_BLOCO':'bloco_economico'}, inplace=True)
#cria o df de bloco geagroficos
dfbloc = dfbloc.loc[:,['CO_PAIS', 'NO_BLOCO']]
#renomear colunas
dfbloc.rename(columns={'CO_PAIS': 'cod_pais','NO_BLOCO':'bloco'}, inplace=True)

In [ ]:
# cod[11] e cod[12] fazer uma para o nome dos estados e região, e juntar ao nome do município da empresa?

In [ ]:
# cod[13]  é a tradução da via
# cod[14] é o nome dos urf

##juntas as classificaçõe em único df

In [ ]:
#juntar os cinco
# df_class = [df_sh, df_cuci, df_cgce, df_isic, df_siit]

In [ ]:
df_sh.dtypes

CO_NCM         int64
NO_NCM_POR    object
CO_SH6         int64
NO_SH6_POR    object
NO_SH4_POR    object
NO_SH2_POR    object
NO_SEC_POR    object
dtype: object

In [ ]:
#juntar os df_sh com df_cuci
df12 = pd.merge(df_sh, df_cuci, on='CO_NCM')
df12.shape

(13598, 13)

In [ ]:
#juntar ao anterior o df_cgce
df123 = pd.merge(df12, df_cgce, on='CO_NCM')
df123.shape

(13598, 17)

In [ ]:
#juntar ao anterior o df_isic
df1234 = pd.merge(df123, df_isic, on='CO_NCM')
df1234.shape

(13598, 22)

In [ ]:
#juntar ao antreior o df_siit
df_class = pd.merge(df1234, df_siit, on='CO_NCM')
df_class.shape

(13322, 23)

In [ ]:
#ver as informações do df pronto após todos os merges
df_class.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13322 entries, 0 to 13321
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CO_NCM           13322 non-null  int64 
 1   NO_NCM_POR       13322 non-null  object
 2   CO_SH6           13322 non-null  int64 
 3   NO_SH6_POR       13322 non-null  object
 4   NO_SH4_POR       13322 non-null  object
 5   NO_SH2_POR       13322 non-null  object
 6   NO_SEC_POR       13322 non-null  object
 7   CO_CUCI_ITEM     13322 non-null  object
 8   NO_CUCI_ITEM     13322 non-null  object
 9   NO_CUCI_SUB      13322 non-null  object
 10  NO_CUCI_GRUPO    13322 non-null  object
 11  NO_CUCI_DIVISAO  13322 non-null  object
 12  NO_CUCI_SEC      13322 non-null  object
 13  CO_CGCE_N3       13322 non-null  int64 
 14  NO_CGCE_N3       13322 non-null  object
 15  NO_CGCE_N2       13322 non-null  object
 16  NO_CGCE_N1       13322 non-null  object
 17  CO_ISIC_CLASSE   13322 non-null

In [ ]:
df_class.tail()

,CO_NCM,NO_NCM_POR,CO_SH6,NO_SH6_POR,NO_SH4_POR,NO_SH2_POR,NO_SEC_POR,CO_CUCI_ITEM,NO_CUCI_ITEM,NO_CUCI_SUB,NO_CUCI_GRUPO,NO_CUCI_DIVISAO,NO_CUCI_SEC,CO_CGCE_N3,NO_CGCE_N3,NO_CGCE_N2,NO_CGCE_N1,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_GRUPO,NO_ISIC_DIVISAO,NO_ISIC_SECAO,NO_SIIT
13317,99997103,Joalheria de ouro do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71",Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...,311,Bens de consumo duráveis – exceto equipamentos...,Bens de consumo duráveis,BENS DE CONSUMO (BC),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS N.C.I.T
13318,99997104,Outros artigos do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71",Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...,322,Bens de consumo não duráveis,Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS N.C.I.T
13319,99999900,Qualquer outra mercadoria sem cobertura cambial,999999,Qualquer outra mercadoria sem cobertura cambia...,Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...,999,Bens não especificados anteriormente,Bens não especificados anteriormente,BENS NÃO ESPECIFICADOS ANTERIORMENTE,8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS N.C.I.T
13320,99999951,Devolução antes da DI (exportação definitiva),999999,Qualquer outra mercadoria sem cobertura cambia...,Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...,999,Bens não especificados anteriormente,Bens não especificados anteriormente,BENS NÃO ESPECIFICADOS ANTERIORMENTE,8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS N.C.I.T
13321,99999960,Reexportação,999999,Qualquer outra mercadoria sem cobertura cambia...,Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES NAO ESPECIFICADAS EM ...,999,Bens não especificados anteriormente,Bens não especificados anteriormente,BENS NÃO ESPECIFICADOS ANTERIORMENTE,8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS N.C.I.T


##esquema para o spark

In [ ]:
df_class.dtypes

ano	mes	mercadoria	ncm_unidade	estado	meio_transporte	urf_localizacao	qnt_estatistica	peso_liquido	valor_dolar	pais

In [ ]:
# Preparar schema para importar dataframe (StructTypes)

esquema = (
    StructType([
        StructField('ncm', IntegerType()),
        StructField('produto', StringType()),
        StructField('sh6', IntegerType()),
        StructField('sh6_mercadoria', StringType()),
        StructField('sh4_posicao', StringType()),
        StructField('sh6_capitulo', StringType()),
        StructField('sh6_secao', StringType()),
        StructField('cuci', StringType()),
        StructField('cuci_item', StringType()),
        StructField('cuci_subgrupo', StringType()),
        StructField('cuci_grupo', StringType()),
        StructField('cuci_divisao', StringType()),
        StructField('cuci_secao', StringType()),
        StructField('cgce', IntegerType()),
        StructField('cgce_economica_n3', StringType()),
        StructField('cgce_n2', StringType()),
        StructField('cgce_n1', StringType()),
        StructField('isic', IntegerType()),
        StructField('isic_class_industrial', StringType()),
        StructField('isic_grupo', StringType()),
        StructField('isic_divisao', StringType()),
        StructField('isic_secao', StringType()),
        StructField('siit_setor_industrial', StringType()),
    ])
)

In [ ]:
# Conversão para PySpark (utilizando schema definido - o que renomeará as colunas)
dfcat = spark.createDataFrame(df_class, schema = esquema)

In [ ]:
dfcat.describe()

DataFrame[summary: string, ncm: string, produto: string, sh6: string, sh6_mercadoria: string, sh4_posicao: string, sh6_capitulo: string, sh6_secao: string, cuci: string, cuci_item: string, cuci_subgrupo: string, cuci_grupo: string, cuci_divisao: string, cuci_secao: string, cgce: string, cgce_economica_n3: string, cgce_n2: string, cgce_n1: string, isic: string, isic_class_industrial: string, isic_grupo: string, isic_divisao: string, isic_secao: string, siit_setor_industrial: string]

In [ ]:
dfcat.show()

+-------+--------------------+-----+--------------------+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+----+---------------------+---------------+--------------------+------------+---------------------+
|    ncm|             produto|  sh6|      sh6_mercadoria|         sh4_posicao| sh6_capitulo|           sh6_secao| cuci|           cuci_item|       cuci_subgrupo|          cuci_grupo|        cuci_divisao|          cuci_secao|cgce|   cgce_economica_n3|             cgce_n2|             cgce_n1|isic|isic_class_industrial|     isic_grupo|        isic_divisao|  isic_secao|siit_setor_industrial|
+-------+--------------------+-----+--------------------+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------

In [ ]:
#salvar um df spark
dfcat.write.format('csv').mode('overwrite').save('/content')

##voltar pro pandas renomeado

In [ ]:
#converte o dataframe spakr para pandas
df = dfcat.toPandas()
df.shape

(13322, 23)

In [ ]:
df

,ncm,produto,sh6,sh6_mercadoria,sh4_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
0,1011010,Cavalos reprodutores de raça pura,10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
2,1011100,"Cavalos reprodutores, de raça pura",10111,"Cavalos reprodutores, de raça pura","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
3,1011900,"Outros cavalos, vivos",10119,"Outros cavalos, vivos","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,210,"Alimentos e bebidas básicos, destinados princi...","Alimentos e bebidas básicos, destinados princi...",BENS INTERMEDIÁRIOS (BI),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
4,1012000,Asininos e muares vivos,10120,Asininos e muares vivos,"Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13317,99997103,Joalheria de ouro do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71",Consumo de bordo (exceto combustíveis e lubrif...,Transações especiais,Transações especiais,9310,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,Operações especiais e commodities não classifi...,MERCADORIAS E TRANSACOES 

In [ ]:
# df.drop_duplicates()

##salvar e enviar ao bucket

In [ ]:
#salvar
df.to_csv('imp-exp-classificacoes.csv',index=False)

dfpais.to_csv('nome-pais.csv',index=False)

In [ ]:
#salvar as duas bases com os blocos
dfeco.to_csv('bloco-economico.csv',index=False)
dfbloc.to_csv('bloco-geagrofico.csv',index=False)

In [ ]:
#enviar para a bucket
bucket.blob('comex-tratado/imp-exp-classificacoes').upload_from_filename('/content/imp-exp-classificacoes.csv')

In [ ]:
#enviar para a bucket
bucket.blob('comex-tratado/nome-pais').upload_from_filename('/content/nome-pais.csv')

In [ ]:
bucket.blob('comex-tratado/bloco-economico').upload_from_filename('/content/bloco-economico.csv')
bucket.blob('comex-tratado/bloco-geografico').upload_from_filename('/content/bloco-geagrofico.csv')

#Conexão com o Big Query

 Envio do dataframe tratado para BigQuerry (utilizando Pandas) - cod Marcos



In [ ]:
# Chave de acesso ao BigQuery com as seguintes roles: Big Query User e Big Query Data Editor
key_path= "/content/chave_bigquery_JSON.json"

# Variável credentials
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "bc26-ed7-projeto-final"

#enviar base de auxilio

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'imp-exp-classificaçoes')

pandas_gbq.to_gbq(df, 'comercio_brasil.imp-exp-classificacoes', project_id='bc26-ed7-projeto-final', if_exists='replace')

# OK

100%|██████████| 1/1 [00:00<00:00, 6743.25it/s]


In [ ]:
df.shape

(13322, 23)

##enviar a base com os nomes de paises para o big query

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'nome-pais')
pandas_gbq.to_gbq(dfpais, 'comercio_brasil.nome_pais', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 8405.42it/s]


In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'nome-pais')
pandas_gbq.to_gbq(dfblco, 'comercio_brasil.contimente', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 7884.03it/s]


In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'nome-pais')
pandas_gbq.to_gbq(dfecom, 'comercio_brasil.bloco_ecomonico', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 7096.96it/s]


codigos usado no bq

In [ ]:
#SELECT * FROM `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` WHERE sh6_posicao = 'Polímeros de etileno, em formas primárias'

In [ ]:
#SELECT distinct(produto) FROM `bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022` LIMIT 1000

In [ ]:
# spark.sql('SELECT * FROM dfcat WHERE sh6_posicao =="Polímeros de etileno, em formas primárias"')

In [ ]:
#enviar a base com o nome dos paises
pandas_gbq.to_gbq(dfpais, 'comercio_brasil.pais', project_id='bc26-ed7-projeto-final', if_exists='replace')

In [ ]:
#enviar as base de paises para agrupamento
pandas_gbq.to_gbq(dfbloc, 'comercio_brasil.bloco', project_id='bc26-ed7-projeto-final', if_exists='replace')
pandas_gbq.to_gbq(dfeco, 'comercio_brasil.bloco-economico', project_id='bc26-ed7-projeto-final', if_exists='replace')

#3 caminho para as bases tratades em parquet e cvs

In [ ]:
#Caminho das base no bucket
#tratada
path_imp_mun = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
path_imp_prod = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'
path_exp_mun = 'gs://bc26-datasets-comex//comex-tratado/EXP_2012_2022_tratado(MUN).parquet'
path_exp_prod = 'gs://bc26-datasets-comex/comex-tratado/EXP_2012_2022_tratado(NCM).parquet'

#tradato - checar o link
path_comb_preco = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATADO'
path_comb_vol = 'bc26-datasets-comex/combustivel-tratado/vendas-combustiveis-2013-2022' 
#base de categorias
path_cat = 'gs://bc26-datasets-comex/comex-tratado/imp-exp-classificacoes'

In [ ]:
# que base é essa?
path_comex = 'gs://bc26-datasets-comex/bc26-datasets-comex/comex-tratado'

In [ ]:
#Caminho das base no bucket - minha nova versão

#tratada importação
path_imp_mun = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
path_imp_prod = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'

#tratada exportação 
path_exp_mun = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'

path_exp_prod = 'gs://bc26-datasets-comex/comex-tratado/EXP_2013-2022_tratado(NCM).csv'

#tratada combustíveis
path_comb = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATAMENTO-OK'
#tratasa combustívies com volume
path_comb_vol = 'gs://bc26-datasets-comex/combustivel-tratado/volume_preco_combustiveis_mensal_2013_2022' 


#Enviar a base de exportação para o BQ

In [ ]:
#criar o dataframe de exportação por municipios
df_em = pd.read_parquet(path_exp_mun)
df_em.head(2)

,mes,SH4,cod_pais,estado,cod_municipio,peso_liquido,valor_dolar,descricoes__sistema_harmonizado,municipio,ano
0,janeiro,6115,160,RS,4314902,2512,37327,"Polímeros de etileno, em formas primárias",Porto Alegre,2012
1,fevereiro,8481,23,RS,4314902,80,8444,"Fatos de saia-casaco, conjuntos, casacos, vest...",Porto Alegre,2012


In [ ]:
df_em.dtypes


mes                                  object
SH4                                   int64
cod_pais                              int64
estado                               object
cod_municipio                         int64
peso_liquido                          int64
valor_dolar                           int64
descricoes__sistema_harmonizado      object
municipio                            object
ano                                category
dtype: object

In [ ]:
df_em['ano'] = df_em['ano'].astype(int)

In [ ]:
df_em.dtypes

mes                                object
SH4                                 int64
cod_pais                            int64
estado                             object
cod_municipio                       int64
peso_liquido                        int64
valor_dolar                         int64
descricoes__sistema_harmonizado    object
municipio                          object
ano                                 int64
dtype: object

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'exp_2012-2022_tratada_mun')

pandas_gbq.to_gbq(df_em, 'comercio_brasil.exp_2012-2022_tratada_mun', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6423.13it/s]


In [ ]:
#criar o dataframe de exportação por produto, com a nomeclaruta do mercosul
df_ep = pd.read_csv(path_exp_prod)
df_ep.head(2)

,ano,mes,cod_mercadoria,unidade,cod_pais,estado,meio_transporte,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,urf_localizacao,mercadoria
0,2013,Março,30067000,Quilograma líquido,169,SP,Marítima,817800,1447,1447,27538,PORTO DE SANTOS,"Preparações em gel, utilizadas em intervenção ..."
1,2013,Janeiro,30067000,Quilograma líquido,493,SP,Marítima,817800,1512,1512,26435,PORTO DE SANTOS,"Preparações em gel, utilizadas em intervenção ..."


In [ ]:
df_ep.dtypes

ano                 int64
mes                object
cod_mercadoria      int64
unidade            object
cod_pais            int64
estado             object
meio_transporte    object
cod_urf             int64
qnt_estatistica     int64
peso_liquido        int64
valor_dolar         int64
urf_localizacao    object
mercadoria         object
dtype: object

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'exp_2012-2022_tratada_ncm')

pandas_gbq.to_gbq(df_ep, 'comercio_brasil.exp_2012-2022_tratada_ncm', project_id='bc26-ed7-projeto-final', if_exists='replace')

# OK

100%|██████████| 1/1 [00:00<00:00, 7269.16it/s]


In [ ]:
#por capcidade de  tamanho apagar
# del df_em

bases importação

In [ ]:
#criar dataframe com as importações por município
df_im = pd.read_parquet(path_im)
df_im.head(2)

,mes,SH4,cod_pais,estado,cod_municipio,peso_liquido,valor_dolar,descricoes__sistema_harmonizado,municipio,ano
0,janeiro,6115,160,RS,4314902,2512,37327,"Polímeros de etileno, em formas primárias",Porto Alegre,2012
1,fevereiro,8481,23,RS,4314902,80,8444,"Fatos de saia-casaco, conjuntos, casacos, vest...",Porto Alegre,2012


In [ ]:
#criar o dataframe com as importações por produtos
df_ip = pd.read_parquet(path_ip)
df_ip.head(2)

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano
0,março,39095021,10,190,CE,317700,211,211,3600,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
1,junho,39095021,10,245,RS,1017600,50,50,459,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012


base preço do combustivel

In [ ]:
#criar o dataframe com os preço dos combustivies
df_comb = pd.read_parquet(path_comb)
df_comb.head(2)

,data_inicial,data_final,regiao,estado,municipio,produto,num_postos_pesquisados,unidade_medida,preco_medio_revenda,desvio_padrao_revenda,preco_min_revenda,preco_max_revenda,margem_media_revenda,coef_variacao_revenda
0,2012-12-30,2013-01-05,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,ETANOL HIDRATADO,80,R$/l,2.273,0.023,2.19,2.299,0.405,0.01
1,2012-12-30,2013-01-05,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,2.134,0.043,2.09,2.190,0.268,0.02


In [ ]:
# del df_ep

In [ ]:
df_em

base de auxilio

In [ ]:
path_pais = 'gs://bc26-datasets-comex/comex-tratado/nome-pais'
path_bloco = 'gs://bc26-datasets-comex/comex-tratado/bloco-economico'
path_ecom = 'gs://bc26-datasets-comex/comex-tratado/bloco-geografico'

In [ ]:
df_pais = pd.read_csv(path_pais)
df_blco = pd.read_csv(path_bloco)
df_ecom = pd.read_csv(path_ecom)

In [ ]:
df_pais.tail()

,cod_pais,pais
276,994,A Designar
277,995,Bancos Centrais
278,997,Organizações Internacionais
279,998,Sem informação
280,999,Não Declarados


#filtro de combustivel na bases de exp/imp produtos

Carregamento das bases

In [ ]:
df_comb = pd.read_parquet(path_comb)

In [ ]:
dfcat = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_cat))

In [ ]:
df = pd.read_csv(path_cat)

##procurar os ncm e sh para combustiveis.

In [ ]:
combustiveis = pd.unique(df_comb.produto)

In [ ]:
print(combustiveis)

['ETANOL HIDRATADO' 'GASOLINA COMUM' 'GLP' 'GNV' 'OLEO DIESEL'
 'OLEO DIESEL S10' 'GASOLINA ADITIVADA']


In [ ]:
# dfcat.show(2)
df.head(2)

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
0,1011010,Cavalos reprodutores de raça pura,10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos","Animais vivos, não incluído pescados ou crustá...",Animais vivos não incluídos no capítulo 03,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T


In [ ]:
filtro_comb_ncm = df.produto.str.contains('etanol | gasolina | GLP | GNV | diesel')
filtro_comb_sh = df.sh6_mercadoria.str.contains('etanol | gasolina | GLP | GNV | diesel') 
#resultado muito amplo

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1864,26202100,"Lamas de gasolina contendo chumbo, etc.",262021,Lamas de gasolina e de compostos antidetonante...,Cinzas e resíduos (exceto provenientes da fabr...,"Minerios, escórias e cinzas",Produtos minerais,2881,Cinzas e resíduos (exceto os provenientes da f...,Cinzas e resíduos (exceto os provenientes da f...,Resíduos de metais de base não ferrosos e de s...,Minérios metálicos e sucata,"MATERIAS EM BRUTO, NAO COMESTIVEIS, EXCETO COM...",230,Insumos industriais básicos,Insumos industriais básicos,BENS INTERMEDIÁRIOS (BI),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1916,27100041,Gasóleo (óleo diesel),271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1951,27101921,Gasóleo (óleo diesel),271019,Outros óleos de petróleo ou de minerais betumi...,"Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2724,29051100,Metanol (álcool metílico),290511,Metanol (álcool metílico),Álcoois acíclicos e seus derivados halogenados...,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51211,Metanol (álcool metílico),Álcoois monohídricos acíclicos,"Álcoois, fenóis, fenóis-álcoois, e seus deriva...",Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2856,29094100,"2,2'-Oxidietanol (dietilenoglicol)",290941,"2,2-Oxidietanol (dietilenoglicol)","Éteres, éteres-álcoois, éteres-fenóis, éteres-...",Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51617,"Éteres-álcoois, éteres-fenóis, éteres-álcoois-...","Éteres, peróxidos de álcoois, peróxidos de éte...",Outros produtos químicos orgânicos,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
...,...,...,...,...,...,...,

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1864,26202100,"Lamas de gasolina contendo chumbo, etc.",262021,Lamas de gasolina e de compostos antidetonante...,Cinzas e resíduos (exceto provenientes da fabr...,"Minerios, escórias e cinzas",Produtos minerais,2881,Cinzas e resíduos (exceto os provenientes da f...,Cinzas e resíduos (exceto os provenientes da f...,Resíduos de metais de base não ferrosos e de s...,Minérios metálicos e sucata,"MATERIAS EM BRUTO, NAO COMESTIVEIS, EXCETO COM...",230,Insumos industriais básicos,Insumos industriais básicos,BENS INTERMEDIÁRIOS (BI),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2724,29051100,Metanol (álcool metílico),290511,Metanol (álcool metílico),Álcoois acíclicos e seus derivados halogenados...,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51211,Metanol (álcool metílico),Álcoois monohídricos acíclicos,"Álcoois, fenóis, fenóis-álcoois, e seus deriva...",Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2856,29094100,"2,2'-Oxidietanol (dietilenoglicol)",290941,"2,2-Oxidietanol (dietilenoglicol)","Éteres, éteres-álcoois, éteres-fenóis, éteres-...",Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51617,"Éteres-álcoois, éteres-fenóis, éteres-álcoois-...","Éteres, peróxidos de álcoois, peróxidos de éte...",Outros produtos químicos orgânicos,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9864,84081010,"Motores diesel/semidiesel, para embarcação, ti...",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2811,"Fabricação de motores e turbinas, exceto aeron...",Fabricação de máquinas de uso geral,Fabricação de máquinas e equipamentos n.c,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9865,84081090,"Outros motores diesel/semidiesel, para embarcação",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2811,"Fabricação de motores e turbinas, exceto aeron...",Fabricação de máquinas de uso geral,Fabricação de máquinas e equipamentos n.c,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9866,84082010,"Motores diesel/semidiesel

In [ ]:
filtro_comb_ncm = df.produto.str.contains('etanol')
filtro_comb_sh = df.sh6_mercadoria.str.contains('etanol')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
2724,29051100,Metanol (álcool metílico),290511,Metanol (álcool metílico),Álcoois acíclicos e seus derivados halogenados...,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51211,Metanol (álcool metílico),Álcoois monohídricos acíclicos,"Álcoois, fenóis, fenóis-álcoois, e seus deriva...",Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2785,29062910,2-feniletanol,290629,Outros álcoois cíclicos aromáticos e seus deri...,"Álcoois cíclicos e seus derivados halogenados,...",Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51235,Álcoois cíclicos aromáticos e seus derivados h...,"Álcoois cíclicos e seus derivados halogenados,...","Álcoois, fenóis, fenóis-álcoois, e seus deriva...",Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2856,29094100,"2,2'-Oxidietanol (dietilenoglicol)",290941,"2,2-Oxidietanol (dietilenoglicol)","Éteres, éteres-álcoois, éteres-fenóis, éteres-...",Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51617,"Éteres-álcoois, éteres-fenóis, éteres-álcoois-...","Éteres, peróxidos de álcoois, peróxidos de éte...",Outros produtos químicos orgânicos,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3380,29221100,Monoetanolamina e seus sais,292211,Monoetanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de funções oxigenadas,Compostos de função nitrogênio,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3381,29221200,Dietanolamina e seus sais,292212,Dietanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de funções oxigenadas,Compostos de função nitrogênio,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3382,29221310,Trietanolamina,292213,Trietanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, 

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
2724,29051100,Metanol (álcool metílico),290511,Metanol (álcool metílico),Álcoois acíclicos e seus derivados halogenados...,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51211,Metanol (álcool metílico),Álcoois monohídricos acíclicos,"Álcoois, fenóis, fenóis-álcoois, e seus deriva...",Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
2856,29094100,"2,2'-Oxidietanol (dietilenoglicol)",290941,"2,2-Oxidietanol (dietilenoglicol)","Éteres, éteres-álcoois, éteres-fenóis, éteres-...",Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51617,"Éteres-álcoois, éteres-fenóis, éteres-álcoois-...","Éteres, peróxidos de álcoois, peróxidos de éte...",Outros produtos químicos orgânicos,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3380,29221100,Monoetanolamina e seus sais,292211,Monoetanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de funções oxigenadas,Compostos de função nitrogênio,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3381,29221200,Dietanolamina e seus sais,292212,Dietanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de funções oxigenadas,Compostos de função nitrogênio,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3382,29221310,Trietanolamina,292213,Trietanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de funções oxigenadas,Compostos de função nitrogênio,Produtos químicos orgânicos,"PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2011,Fabricação de produtos químicos básicos,"Fabricação de produtos químicos básicos, ferti...",Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
3383,29221320,Sais de trietanolamina,292213,Trietanolamina e seus sais,Compostos aminados de funções oxigenadas,Produtos químicos orgânicos,Produtos das indústrias químicas ou indústrias...,51461,"Amino-álcoois, éteres e ésteres (exceto os que...",Compostos amino de fun

Filtro = ETANOL.

29051100	Metanol (álcool metílico)	290511	Metanol (álcool metílico)	Álcoois acíclicos e seus derivados halogenados...	Produtos químicos orgânicos	Produtos das indústrias químicas ou indústrias...	51211	Metanol (álcool metílico)	Álcoois monohídricos acíclicos	Álcoois, fenóis, fenóis-álcoois, e seus deriva...	Produtos químicos orgânicos	PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.	240	Insumos industriais elaborados	Insumos industriais elaborados	BENS INTERMEDIÁRIOS (BI)	2011	Fabricação de produtos químicos básicos	Fabricação de produtos químicos básicos, ferti...	Fabricação de produtos químicos	Indústria de Transformação	PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI..

In [ ]:
filtro_comb_ncm = df.produto.str.contains('gasolina')
filtro_comb_sh = df.sh6_mercadoria.str.contains('gasolina')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1864,26202100,"Lamas de gasolina contendo chumbo, etc.",262021,Lamas de gasolina e de compostos antidetonante...,Cinzas e resíduos (exceto provenientes da fabr...,"Minerios, escórias e cinzas",Produtos minerais,2881,Cinzas e resíduos (exceto os provenientes da f...,Cinzas e resíduos (exceto os provenientes da f...,Resíduos de metais de base não ferrosos e de s...,Minérios metálicos e sucata,"MATERIAS EM BRUTO, NAO COMESTIVEIS, EXCETO COM...",230,Insumos industriais básicos,Insumos industriais básicos,BENS INTERMEDIÁRIOS (BI),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1913,27100029,Outras gasolinas,271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",420,Gasolinas para automóvel (motor spirit) – indí...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1936,27101159,Outras gasolinas,271011,Óleos leves de petróleo ou de minerais betumin...,"Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",420,Gasolinas para automóvel (motor spirit) – indí...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1946,27101259,"Outras gasolinas, exceto para aviação",271012,Óleos leves e preparações,"Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",420,Gasolinas para automóvel (motor spirit) – indí...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9934,84133010,"Bombas para gasolina ou álcool, próprias para ...",841330,"Bombas para combustíveis, lubrificantes ou líq...","Bombas para líquidos, mesmo com dispositivo me...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",7422,"Combustíveis, lubrificantes ou líquidos de arr...","Combustíveis, lubrificantes ou líquidos de arr...","Bombas para líquidos, elevadores de líquidos e...","Máquinas em geral e equipamentos industriais, ...",MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2813,"F

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1864,26202100,"Lamas de gasolina contendo chumbo, etc.",262021,Lamas de gasolina e de compostos antidetonante...,Cinzas e resíduos (exceto provenientes da fabr...,"Minerios, escórias e cinzas",Produtos minerais,2881,Cinzas e resíduos (exceto os provenientes da f...,Cinzas e resíduos (exceto os provenientes da f...,Resíduos de metais de base não ferrosos e de s...,Minérios metálicos e sucata,"MATERIAS EM BRUTO, NAO COMESTIVEIS, EXCETO COM...",230,Insumos industriais básicos,Insumos industriais básicos,BENS INTERMEDIÁRIOS (BI),8999,Não classificado,Não classificado,Não classificado,Outros Produtos,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1910,27100011,Naftas para petroquímica,271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1911,27100019,Outras naftas,271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1912,27100021,Gasolinas de aviação,271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1913,27100029,Outras gasolinas,271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",420,Gasolinas para automóvel (motor spirit) – indí...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fab

FILTRO = GASOLINA, DIESEL
2710---- Gasóleo (óleo diesel)	271000
Óleos de petróleo ou de minerais betuminosos, ...	Combustíveis minerais, óleos minerais e produt...	Produtos minerais	3346	Óleos de petróleo ou de minerais betuminosos (...	Óleos de petróleo ou de minerais betuminosos (...	Óleos combustíveis de petróleo ou de minerais ...	Petróleo, produtos petrolíferos e materiais re...	COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...	421	Combustíveis e lubrificantes elaborados - exce...	Combustíveis e lubrificantes elaborados	COMBUSTÍVEIS E LUBRIFICANTES	1920	Fabricação de produtos petrolíferos refinados	Fabricação de produtos petrolíferos refinados	Fabricação de coque e produtos petrolíferos re...	Indústria de Transformação	PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI..

In [ ]:
filtro_comb_ncm = df.produto.str.contains('glp|GLP')
filtro_comb_sh = df.sh6_mercadoria.str.contains('glp|GLP')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1969,27111910,Gás liquefeito de petróleo (glp),271119,Outros gases liquefeitos de hidrocarbonetos ga...,Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3442,"Hidrocarbonetos gasosos, liquefeitos, n.e.p.","Outros hidrocarbonetos gasosos, liquefeitos",Gás de petróleo e outros hidrocarbonetos gasosos,"Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
6392,39233010,Recipientes para gás liquefeito de petróleo (GLP),392330,"Garrafões, garrafas, frascos, artigos semelhan...","Artigos de transporte ou de embalagem, de plás...",Plásticos e suas obras,Plásticos e suas obras; Borracha e suas obras,89319,"Artigos de transporte ou de embalagem, n.e.p.;...","Artigos de transporte ou de embalagem, de plás...",Outros artigos de plásticos,"Artigos manufaturados diversos, n.e.p.",OBRAS DIVERSAS,240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2220,Fabricação de produtos plásticos,Fabricação de produtos plásticos,Fabricação de produtos de borracha e plásticos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...


In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial


FILTRO = GLP
27111910	Gás liquefeito de petróleo (glp)	271119	Outros gases liquefeitos de hidrocarbonetos ga...	Gás de petróleo e outros hidrocarbonetos gasosos	Combustíveis minerais, óleos minerais e produt...	Produtos minerais	3442	Hidrocarbonetos gasosos, liquefeitos, n.e.p.	Outros hidrocarbonetos gasosos, liquefeitos	Gás de petróleo e outros hidrocarbonetos gasosos	Gás, natural e manufaturado	COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...	421	Combustíveis e lubrificantes elaborados - exce...	Combustíveis e lubrificantes elaborados	COMBUSTÍVEIS E LUBRIFICANTES	1920	Fabricação de produtos petrolíferos refinados	Fabricação de produtos petrolíferos refinados	Fabricação de coque e produtos petrolíferos re...	Indústria de Transformação	PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...

In [ ]:
filtro_comb_ncm = df.produto.str.contains('gnv|GNV|Gás|gás|Gas/gas')
filtro_comb_sh = df.sh6_mercadoria.str.contains('gnv|GNV|Gás|gás|Gas/gas')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
962,8121000,Cerejas conservadas transitoriamente (por exem...,81210,"Cerejas conservadas transitoriamente, mas impr...",Frutas conservadas transitoriamente (por exemp...,Frutas; cascas de frutos cítricos e de melões,Produtos do reino vegetal,05821,Frutas conservadas transitoriamente (por exemp...,"Frutas conservadas transitoriamente, cascas de...","Frutas, preservados e preparações (exceto suco...",Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,210,"Alimentos e bebidas básicos, destinados princi...","Alimentos e bebidas básicos, destinados princi...",BENS INTERMEDIÁRIOS (BI),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
964,8129000,Outras frutas conservadas transitoriamente (po...,81290,"Outras frutas conservadas transitoriamente, ma...",Frutas conservadas transitoriamente (por exemp...,Frutas; cascas de frutos cítricos e de melões,Produtos do reino vegetal,05821,Frutas conservadas transitoriamente (por exemp...,"Frutas conservadas transitoriamente, cascas de...","Frutas, preservados e preparações (exceto suco...",Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,210,"Alimentos e bebidas básicos, destinados princi...","Alimentos e bebidas básicos, destinados princi...",BENS INTERMEDIÁRIOS (BI),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1891,27050000,"Gás de hulha, gás de água, gás pobre (gás de a...",270500,"Gás de hulha, gás de água, gás pobre e gases s...","Gás de hulha, gás de água, gás pobre (gás de a...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3450,"Gás de carvão, gás de água, gás de produção e ...","Gás de carvão, gás de água, gás de produção e ...","Gás de carvão, gás de água, gás de produção e ...","Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,3520,Fabricação de gás; distribuição de combustívei...,Fabricação de gás; distribuição de combustívei...,"Fornecimento de eletricidade, gás, vapor e ar ...",Outros Produtos,PRODUTOS N.C.I.T
1964,27111100,Gás natural liquefeito,271111,"Gás natural, liquefeito",Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3431,"Gás natural, liquefeito","Gás natural, liquefeito","Gás natural, liquefeito ou não","Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",410,Combustíveis e lubrificantes básicos,Combustíveis e lubrificantes básicos,COMBUSTÍVEIS E LUBRIFICANTES,620,Extração de gás natural,Extração de gás natural,Extração de petróleo bruto e gás natural,Indústria Extrativa,PRODUTOS N.C.I.T
1969,27111910,Gás liquefeito de petróleo (glp),271119,Outros gases liquefeitos de hidrocarbonetos ga...,Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3442,"Hidrocarbonetos gasosos, liquefeitos, n.e.p.","Outros hidrocarbonetos gasosos, liquefeitos",Gás de petróleo e outros hidrocarbonetos gasosos,"Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de 

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1891,27050000,"Gás de hulha, gás de água, gás pobre (gás de a...",270500,"Gás de hulha, gás de água, gás pobre e gases s...","Gás de hulha, gás de água, gás pobre (gás de a...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3450,"Gás de carvão, gás de água, gás de produção e ...","Gás de carvão, gás de água, gás de produção e ...","Gás de carvão, gás de água, gás de produção e ...","Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,3520,Fabricação de gás; distribuição de combustívei...,Fabricação de gás; distribuição de combustívei...,"Fornecimento de eletricidade, gás, vapor e ar ...",Outros Produtos,PRODUTOS N.C.I.T
1964,27111100,Gás natural liquefeito,271111,"Gás natural, liquefeito",Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3431,"Gás natural, liquefeito","Gás natural, liquefeito","Gás natural, liquefeito ou não","Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",410,Combustíveis e lubrificantes básicos,Combustíveis e lubrificantes básicos,COMBUSTÍVEIS E LUBRIFICANTES,620,Extração de gás natural,Extração de gás natural,Extração de petróleo bruto e gás natural,Indústria Extrativa,PRODUTOS N.C.I.T
1971,27112100,Gás natural no estado gasoso,271121,Gás natural no estado gasoso,Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3432,"Gás natural, na forma gasosa","Gás natural, na forma gasosa","Gás natural, liquefeito ou não","Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",410,Combustíveis e lubrificantes básicos,Combustíveis e lubrificantes básicos,COMBUSTÍVEIS E LUBRIFICANTES,620,Extração de gás natural,Extração de gás natural,Extração de petróleo bruto e gás natural,Indústria Extrativa,PRODUTOS N.C.I.T
1972,27112910,Butanos no estado gasoso,271129,"Butanos, outros hidrocarbonetos e gás petróleo...",Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3449,"Hidrocarbonetos gasosos em estado gasoso, n.e.p.",Outros hidrocarbonetos gasosos em estado gasoso,Gás de petróleo e outros hidrocarbonetos gasosos,"Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",410,Combustíveis e lubrificantes básicos,Combustíveis e lubrificantes básicos,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1973,27112990,"Outros hidrocarbonetos gasosos e gás petróleo,...",271129,"Butanos, outros hidrocarbonetos e gás petróleo...",Gás de petróleo e outros hidrocarbonetos gasosos,"Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3449,"Hidrocarbonetos gasosos em estado gasoso, n.e.p.",Outros hidrocarbonetos gasosos em estado gasoso,Gás de petróleo e outros hidrocarbonetos gasosos,"Gás, natural e manufaturado","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",410,Combustíveis e lubrificantes básicos,Combustíveis e lubrificantes básicos,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9223,73042200,"Hastes de perfuração de aço inoxidável, dos ti...",730422,"Tubos de perfuração, sem costura, de aços inox...","Tubos e p

27050000	Gás de hulha, gás de água, gás pobre (gás de a...	270500	Gás de hulha, gás de água, gás pobre e gases s...	Gás de hulha, gás de água, gás pobre (gás de a...

27111100	Gás natural liquefeito	271111	Gás natural, liquefeito	Gás de petróleo e outros hidrocarbonetos gasosos

27111910	Gás liquefeito de petróleo (glp)	271119	Outros gases liquefeitos de hidrocarbonetos ga...	Gás de petróleo e outros hidrocarbonetos gasosos

27112100	Gás natural no estado gasoso	271121	Gás natural no estado gasoso	Gás de petróleo e outros hidrocarbonetos gasosos

27112990	Outros hidrocarbonetos gasosos e gás petróleo,...	271129	Butanos, outros hidrocarbonetos e gás petróleo...	Gás de petróleo e outros hidrocarbonetos gasosos




In [ ]:
filtro_comb_ncm = df.produto.str.contains('diesel')
filtro_comb_sh = df.sh6_mercadoria.str.contains('diesel')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1916,27100041,Gasóleo (óleo diesel),271000,"Naftas, gasolinas, querosenes, gasóleo, outros...","Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
1951,27101921,Gasóleo (óleo diesel),271019,Outros óleos de petróleo ou de minerais betumi...,"Óleos de petróleo ou de minerais betuminosos, ...","Combustíveis minerais, óleos minerais e produt...",Produtos minerais,3346,Óleos de petróleo ou de minerais betuminosos (...,Óleos de petróleo ou de minerais betuminosos (...,Óleos combustíveis de petróleo ou de minerais ...,"Petróleo, produtos petrolíferos e materiais re...","COMBUSTIVEIS MINERAIS, LUBRIFICANTES E MATERIA...",421,Combustíveis e lubrificantes elaborados - exce...,Combustíveis e lubrificantes elaborados,COMBUSTÍVEIS E LUBRIFICANTES,1920,Fabricação de produtos petrolíferos refinados,Fabricação de produtos petrolíferos refinados,Fabricação de coque e produtos petrolíferos re...,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
6038,38260000,"Biodiesel e suas misturas, que não contenham o...",382600,"Biodiesel e suas misturas, com > 70% peso de p...","Biodiesel e suas misturas, que não contenham o...",Produtos diversos das indústrias químicas,Produtos das indústrias químicas ou indústrias...,59899,Outros produtos e preparações químicas,Outros produtos químicos e preparações,Outros produtos diversos das indústrias químicas,"Materiais e produtos químicos, n.e.p.","PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2029,Fabricação de outros produtos químicos n.c,Fabricação de outros produtos químicos,Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9864,84081010,"Motores diesel/semidiesel, para embarcação, ti...",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2811,"Fabricação de motores e turbinas, exceto aeron...",Fabricação de máquinas de uso geral,Fabricação de máquinas e equipamentos n.c,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9865,84081090,"Outros motores diesel/semidiesel, para embarcação",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para 

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
6038,38260000,"Biodiesel e suas misturas, que não contenham o...",382600,"Biodiesel e suas misturas, com > 70% peso de p...","Biodiesel e suas misturas, que não contenham o...",Produtos diversos das indústrias químicas,Produtos das indústrias químicas ou indústrias...,59899,Outros produtos e preparações químicas,Outros produtos químicos e preparações,Outros produtos diversos das indústrias químicas,"Materiais e produtos químicos, n.e.p.","PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.",240,Insumos industriais elaborados,Insumos industriais elaborados,BENS INTERMEDIÁRIOS (BI),2029,Fabricação de outros produtos químicos n.c,Fabricação de outros produtos químicos,Fabricação de produtos químicos,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9864,84081010,"Motores diesel/semidiesel, para embarcação, ti...",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2811,"Fabricação de motores e turbinas, exceto aeron...",Fabricação de máquinas de uso geral,Fabricação de máquinas e equipamentos n.c,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9865,84081090,"Outros motores diesel/semidiesel, para embarcação",840810,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71333,Motores de pistão de combustão interna de igni...,"Motores de pistão, de propulsão marítima","Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2811,"Fabricação de motores e turbinas, exceto aeron...",Fabricação de máquinas de uso geral,Fabricação de máquinas e equipamentos n.c,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9866,84082010,"Motores diesel/semidiesel, para vículos do cap...",840820,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71323,De ignição por compressão dos motores de pistã...,Motores de pistão para a propulsão de veículos...,"Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipamentos de transporte,Peças para equipamentos de transporte,BENS INTERMEDIÁRIOS (BI),2910,Fabricação de veículos automotores,Fabricação de veículos automotores,"Fabricação de veículos automóveis, reboques e ...",Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI...
9867,84082020,"Motores diesel/semidiesel, para veículos do ca...",840820,"Motores de pistão, de ignição por compressão, ...","Motores de pistão, de ignição por compressão (...","Reatores nucleares, caldeiras, máquinas, apare...","Máquinas e aparelhos, material elétrico e suas...",71323,De ignição por compressão dos motores de pistã...,Motores de pistão para a propulsão de veículos...,"Motores de pistão, e suas partes",Máquinas e equipamentos de geração de energia,MAQUINAS E EQUIPAMENTOS DE TRANSPORTE,280,Peças para equipa



FILTRO = DIESEL
38260000	Biodiesel e suas misturas, que não contenham o.. 

Biodiesel e suas misturas, que não contenham o...	Produtos diversos das indústrias químicas	Produtos das indústrias químicas ou indústrias...	59899	Outros produtos e preparações químicas	Outros produtos químicos e preparações	Outros produtos diversos das indústrias químicas	Materiais e produtos químicos, n.e.p.	PRODUTOS QUIMICOS E RELACIONADOS, N.E.P.	240	Insumos industriais elaborados	Insumos industriais elaborados	BENS INTERMEDIÁRIOS (BI)	2029	Fabricação de outros produtos químicos n.c	Fabricação de outros produtos químicos	Fabricação de produtos químicos	Indústria de Transformação	PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE MEDI.

In [ ]:
filtro_comb_ncm = df.produto.str.contains('álcool|Álcool')
filtro_comb_sh = df.sh6_mercadoria.str.contains('álcool|Álcool')

In [ ]:
df.loc[filtro_comb_ncm]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1544,20091100,"Suco (sumo) de laranja, não fermentados, sem a...",200911,"Sucos de laranjas, congelados, não fermentados",Sumos de frutas (incluídos os mostos de uvas) ...,"Preparações de produtos hortícolas, de frutas ...","Produtos das indútrias alimentares; Bebidas, l...",0591,Suco de laranja,Suco de laranja,Sucos de frutas ou de vegetais,Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1545,20091200,"Suco (sumo) de laranja, não fermentados, sem a...",200912,"Sucos de laranja não congelados, não fermentad...",Sumos de frutas (incluídos os mostos de uvas) ...,"Preparações de produtos hortícolas, de frutas ...","Produtos das indútrias alimentares; Bebidas, l...",0591,Suco de laranja,Suco de laranja,Sucos de frutas ou de vegetais,Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1546,20091900,"Outros sucos de laranjas, não fermentados, sem...",200919,"Outros sucos de laranjas, não fermentados",Sumos de frutas (incluídos os mostos de uvas) ...,"Preparações de produtos hortícolas, de frutas ...","Produtos das indútrias alimentares; Bebidas, l...",0591,Suco de laranja,Suco de laranja,Sucos de frutas ou de vegetais,Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1574,20099000,"Misturas de sucos (sumos), não fermentados, se...",200990,"Misturas de sucos, não fermentados",Sumos de frutas (incluídos os mostos de uvas) ...,"Preparações de produtos hortícolas, de frutas ...","Produtos das indútrias alimentares; Bebidas, l...",05996,Misturas de sucos de frutas ou de produtos hor...,Suco de qualquer fruta (exceto citrinos) ou ve...,Sucos de frutas ou de vegetais,Vegetais e frutas,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1030,Processamento e conservação de frutas e legumes,Processamento e conservação de frutas e legumes,Fabricação de produtos alimentícios,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1619,22029100,Cerveja sem álcool,220291,Cerveja sem álcool,"Águas, incluídas as águas minerais e as águas ...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11102,Águas (incluídas as águas minerais e as águas ...,Outras bebidas não-alcoólicas,Outras bebidas não-alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1104,Fabricação de refrigerantes; produção de águas...,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1620,22029900,"Outras bebidas não-alcoólicas (exceto água, ce...",220299,"Outras bebidas não-alcoólicas (exc

In [ ]:
df.loc[filtro_comb_sh]

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,cuci_grupo,cuci_divisao,cuci_secao,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
1619,22029100,Cerveja sem álcool,220291,Cerveja sem álcool,"Águas, incluídas as águas minerais e as águas ...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11102,Águas (incluídas as águas minerais e as águas ...,Outras bebidas não-alcoólicas,Outras bebidas não-alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1104,Fabricação de refrigerantes; produção de águas...,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1620,22029900,"Outras bebidas não-alcoólicas (exceto água, ce...",220299,"Outras bebidas não-alcoólicas (exceto água, ce...","Águas, incluídas as águas minerais e as águas ...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11102,Águas (incluídas as águas minerais e as águas ...,Outras bebidas não-alcoólicas,Outras bebidas não-alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1104,Fabricação de refrigerantes; produção de águas...,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1624,22042100,"Outros vinhos, mostos de uvas, fermentados, im...",220421,"Outros vinhos; mostos de uvas, cuja fermentaçã...","Vinhos de uvas frescas, incluídos os vinhos en...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11217,Vinhos de uvas frescas (exceto vinho espumante...,Vinhos de uvas frescas (incluindo vinho fortif...,Bebidas alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1102,Fabricação de vinhos,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1625,22042211,Vinhos em recipientes de capacidade não superi...,220422,Outros vinhos; mostos de uvas cuja fermentação...,"Vinhos de uvas frescas, incluídos os vinhos en...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11217,Vinhos de uvas frescas (exceto vinho espumante...,Vinhos de uvas frescas (incluindo vinho fortif...,Bebidas alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1102,Fabricação de vinhos,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1626,22042219,Vinhos em recipientes de capacidade superior a...,220422,Outros vinhos; mostos de uvas cuja fermentação...,"Vinhos de uvas frescas, incluídos os vinhos en...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11217,Vinhos de uvas frescas (exceto vinho espumante...,Vinhos de uvas frescas (incluindo vinho fortif...,Bebidas alcoólicas,Bebidas,BEBIDAS E TABACO,324,"Alimentos e bebidas elaborados, destinados pri...",Bens de consumo semiduráveis e não duráveis,BENS DE CONSUMO (BC),1102,Fabricação de vinhos,Fabricação de bebidas,Fabricação de bebidas,Indústria de Transformação,PRODUTOS DA INDUSTRIA DE TRANSFORMAÇÃO DE BAIX...
1627,22042220,"Mostos de uvas, em recipientes de capacidade s...",220422,Outros vinhos; mostos de uvas cuja fermentação...,"Vinhos de uvas frescas, incluídos os vinhos en...","Bebidas, líquidos alcoólicos e vinagres","Produtos das indútrias alimentares; Bebidas, l...",11217,Vinhos de uvas frescas (exceto vinho espumante...,Vinhos de 

##lista de interrese

In [ ]:
ncm	      produto	          sh6	sh6_mercadoria	sh4_posicao	
27100029	Outras gasolinas	271000	Naftas, gasolinas, querosenes, gasóleo, outros...	Óleos de petróleo ou de minerais betuminosos, ...
27100041	Gasóleo (óleo diesel)	271000	Naftas, gasolinas, querosenes, gasóleo, outros...	Óleos de petróleo ou de minerais betuminosos, ...
27100042	Fuel-oil (óleo combustivel)	271000	Naftas, gasolinas, querosenes, gasóleo, outros...	Óleos de petróleo ou de minerais betuminosos, ...
27100049	Outros óleos combustíveis	271000	Naftas, gasolinas, querosenes, gasóleo, outros...	Óleos de petróleo ou de minerais betuminosos, ...
27101159	Outras gasolinas	271011	Óleos leves de petróleo ou de minerais betumin...	Óleos de petróleo ou de minerais betuminosos, ...
27101259	Outras gasolinas, exceto para aviação	271012	Óleos leves e preparações	Óleos de petróleo ou de minerais betuminosos, ...
27101921	Gasóleo (óleo diesel)	271019	Outros óleos de petróleo ou de minerais betumi...	Óleos de petróleo ou de minerais betuminosos, ...
27111100	Gás natural liquefeito	271111	Gás natural, liquefeito	Gás de petróleo e outros hidrocarbonetos gasosos
27111910	Gás liquefeito de petróleo (glp)	271119	Outros gases liquefeitos de hidrocarbonetos ga...	Gás de petróleo e outros hidrocarbonetos gasosos
27112100	Gás natural no estado gasoso	271121	Gás natural no estado gasoso	Gás de petróleo e outros hidrocarbonetos gasosos
27112990	Outros hidrocarbonetos gasosos e gás petróleo,...	271129	Butanos, outros hidrocarbonetos e gás petróleo...	Gás de petróleo e outros hidrocarbonetos gasosos
29051100  Metanol (álcool metílico) 290511 Metanol (álcool metílico) Álcoois acíclicos e seus derivados halogenados...
38260000	Biodiesel e suas misturas, que não contenham o...	382600	Biodiesel e suas misturas, com > 70% peso de p...	Biodiesel e suas misturas, que não contenham o...


##procurar esse códigos na bases completas de importação e exportação

In [ ]:
sh6 = [271000, 271011, 271012,271019,271111,271119,271121,271129,290511,382600]
ncm = [27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000]
#o sh4 é inclui algumas variações
sh4 = [2710, 2711, 2905, 3826]

In [ ]:
#ver para esses ncm qual é o cuci!

#4 - analise spark

feita em outro colab
<a href="https://colab.research.google.com/drive/1lqZ8jtAtEbTtEfDtkLD4EBGSjWkU9mM-#scrollTo=326clQ-PKkj7">Comex-pyspark</a>

na VM-colab

In [ ]:
#mortar o dataframe no pyspark, usando o esquema
def montar_df(caminho, formato):
  df = ( spark.read.format(formato)
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(caminho))
  return df


In [ ]:
#chamar a função para criar a base com o esquema
# dfpais = montar_df(path_pais, 'csv')

In [ ]:
# dfip = montar_df(path_ip, 'parquet')

In [ ]:
df_pais
dfpais = spark.createDataFrame(df_pais)

In [ ]:
dfpais.describe().show()

+-------+-----------------+----------+
|summary|          CO_PAIS|   NO_PAIS|
+-------+-----------------+----------+
|  count|              281|       281|
|   mean|449.4697508896797|      null|
| stddev| 266.870822101005|      null|
|    min|                0|A Designar|
|    max|              999|     Índia|
+-------+-----------------+----------+



In [ ]:
dfim = spark.createDAtaFrame(df_im)

In [ ]:
dfim.show()

base importação por produto.

In [ ]:
#covertar a de pandas para pyspark
dfip = spark.createDataFrame(df_ip)

In [ ]:
#pegando direto do bucket em spark
dfip = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_ip))

demorou 19min

In [ ]:
#estorou o tamanho da memória.
dfip.show()

In [ ]:
dfip.printSchema()

In [ ]:
dfip.describe().show()

In [ ]:
dfip.summary().show()

In [ ]:
#os 10 produtos importados com maior valor em dolares no 10 anos.
dfip.select(F.col('produto')).orderBy(F.col('valor_dolar').desc()).show(10)

In [ ]:
#agrupando por ano
dfip.select(F.col('produto')).groupBy(F.col('ano')).orderBy(F.col('valor_dolar').desc()).show()
#faz o que quero?

In [ ]:
#os 10 produtos importados com maior volume no 10 anos.
dfip.select(F.col('produto')).orderBy(F.col('peso_liquido').desc()).show(10)

In [ ]:
#agupando por ano
dfip.select(F.col('produto')).groupBy(F.col('ano')).orderBy(F.col('peso_liquido').desc()).show()

In [ ]:
#criar coluna com o valor das importação por volume transportado
dfip = dfip.withColumn('dolar_por_vol', F.round(F.col('valor_dolar')/F.col('peso_liquido'), 2))

In [ ]:
#os 10 produtos importados com maior valor por volume.
dfip.select(F.col('mercadoria')).orderBy(F.col('dolar_por_vol').desc()).show(10)

#Big Query

##select usado no BQ na importacao-ncm

In [ ]:
SELECT * FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` LIMIT 1000;

#as 10 maiores importação de produtos em 10 anos pelo valor (unica nota)
SELECT produto, valor_dolar FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
ORDER BY valor_dolar DESC LIMIT 10;

#em 10 anos os 10 maiores gastos em produtos
SELECT produto, sum(valor_dolar) as valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY produto ORDER BY sum(valor_dolar) DESC LIMIT 10;

#no ano de 2022, os 10 protudos importados com maior valor gasto nesses produtos
SELECT produto, sum(valor_dolar) AS valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE ano=2022 GROUP BY produto ORDER BY sum(valor_dolar) DESC LIMIT 10;

#as 10 maiores importação de produtos em 10 anos pelo volume em único produto num mês e ano
SELECT distinct(produto), peso_liquido FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
ORDER BY peso_liquido DESC LIMIT 10;

#em 10 anos os 10 produtos com maior volume importado
SELECT produto, sum(peso_liquido) as volume_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY produto ORDER BY sum(peso_liquido) DESC LIMIT 10;

#paises
#10 paises do qual há mais importados de em 10 anos em valor
SELECT p.pais, sum(i.valor_dolar) AS valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais=p.cod_pais
GROUP BY p.pais ORDER BY sum(i.valor_dolar) DESC LIMIT 10;

#10 paises do qual há mais importados de em 10 anos em volume
SELECT p.pais, sum(i.peso_liquido) AS volume_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais=p.cod_pais
GROUP BY p.pais ORDER BY sum(i.peso_liquido) DESC LIMIT 10;

#10 paises do mais foram importados em valor de produtos no ano de 2022
SELECT p.pais, sum(i.valor_dolar) AS valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais=p.cod_pais
WHERE i.ano=2022 GROUP BY p.pais ORDER BY sum(i.valor_dolar) DESC LIMIT 10;

#10 paises do mais foram importados em volume de produtos no ano de 2022
SELECT p.pais, sum(i.peso_liquido) AS valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais=p.cod_pais
WHERE i.ano=2022 GROUP BY p.pais ORDER BY sum(i.peso_liquido) DESC LIMIT 10;

#os 50 maiores produtos valor por volume (tipo dolar/metro cubico) dos produtos nos 10 anos.
#conferir qual é a unidade de volume usada!
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, (sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE peso_liquido !=0 GROUP BY produto ORDER BY dolar_por_volume DESC LIMIT 50;

#os 50 maiores produtos valor por volume dos produtos no ano de 2022
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, (sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE ano=2020 AND peso_liquido !=0
GROUP BY produto ORDER BY dolar_por_volume DESC LIMIT 50;

#as via de transporte com de acordo com o volume importado
SELECT meio_transporte, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY meio_transporte ORDER BY volume_total DESC;

#as via de transporte com de acordo com o volume importado no ano de 2020
SELECT meio_transporte, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE ano=2021
GROUP BY meio_transporte ORDER BY volume_total DESC;

#as via de transporte com de acordo com o valor importado
SELECT meio_transporte, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY meio_transporte ORDER BY valor_total DESC;

#meses e anos ordenado pelo volume total importado
SELECT ano, mes, sum(valor_dolar) as valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY ano, mes ORDER BY valor_total DESC;

#ano com maior valor de importação
SELECT ano, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY ano ORDER BY valor_total DESC;

#ano com maior volume de importação
SELECT ano, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY ano ORDER BY volume_total DESC;

#mes que ocorre mais importação ordenado pelo volume total importado
SELECT mes, sum(valor_dolar) as valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY mes ORDER BY valor_total DESC;



#filtro com os produtos que estão na base de combustívies
SELECT * FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000) LIMIT 1000;

#quantas entradas tem o filtro com produtos da base combustivel
SELECT count(*) FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000);

#quanto foi importado desdes produtos em valor e volume nos 10 anos
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000)
GROUP BY produto;

#acrescentando o valor por volume desses produtos nos 10 anos
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, (sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000)
GROUP BY produto ORDER BY dolar_por_volume DESC;

#acrescentando o valor por volume desses produtos no ano de 2022
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, (sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000)
AND ano=2020 GROUP BY produto ORDER BY dolar_por_volume DESC;


##group by usado na importação ncm

In [ ]:
#testando a sintaxe do inner join
SELECT im.produto, c.siit_setor_industrial 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm LIMIT 1000;

#de acordo com o SIIT
#valor total gasto em importação no 10 anos por setor industrial
SELECT c.siit_setor_industrial, SUM(im.valor_dolar) AS valor_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total;

#valor total gasto em importação no 10 anos em 
SELECT c.siit_setor_industrial, SUM(im.peso_liquido) AS volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY volume_total;

#juntando as duas queyries anterior
#valor total gasto em importação e o volume no 10 anos em relação a cada setor industrial
SELECT c.siit_setor_industrial, SUM(im.valor_dolar) AS valor_total, SUM(im.peso_liquido) AS volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1;

#valor total gasto em importação e o volume em relação a cada setor industrial em 2022
SELECT c.siit_setor_industrial, SUM(im.valor_dolar) AS valor_total, SUM(im.peso_liquido) AS volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
WHERE ano=2022 GROUP BY 1 ORDER BY valor_total;

#calcular percentual (todos os anos da base) valor total de importação
SELECT SUM(valor_dolar) AS valor_total FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`;
#2169120802018 dolares = 2 tri 169 bi 120 milhões de dolares

#média do valor de importação por mes-protudo
SELECT AVG(valor_dolar) AS media_valor FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`;
#106384.478 dolares por produto e mes

#média do volume de importação por mes-protudo
SELECT AVG(peso_liquido) AS media_volume FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`;
#82602.9 kg-l por produto e mes

#em única query ver a soma e média do valor e volume
SELECT SUM(valor_dolar) AS soma_total, AVG(valor_dolar) AS media_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`;
#1684227798719 volume total

#percentual do valor e de volume de importação de cada setor industrial
SELECT c.siit_setor_industrial, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY porcento_valor DESC;

#valor e percentual gasto em importação por setor industrial nos 11 anos
SELECT c.siit_setor_industrial, 
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY porcento_volume DESC;

#o valor e volume total no ano de 2022
SELECT SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`
WHERE ano=2022;
#valor total em 2022(250808584820) e volume total em (158139630507)

#valor e percentual gasto em importação por setor industrial no ano de 2022
SELECT c.siit_setor_industrial, 
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(250808584820) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(158139630507) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm WHERE ano=2022
GROUP BY 1 ORDER BY porcento_volume DESC;

#o valor e volume total em cada ano
SELECT ano, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`
GROUP BY ano ORDER BY ano;

#media e total de importação por valor e volume em cada mês
SELECT mes, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`
GROUP BY mes ORDER BY valor_total;

#no ano de 2022, o valor total, a média e a porcentagem no ano do valor e volume importado
SELECT mes, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
SUM(valor_dolar)*100/(250808584820) AS percentual_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume, 
SUM(peso_liquido)*100/(158139630507) AS percentual_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`
where ano =2022 GROUP BY mes ORDER BY percentual_valor DESC;

#salvar a tabela com o valor agrupado por ano para fazer a porcentagem em cada ano
CREATE OR REPLACE TABLE comercio_brasil.imp_ano AS (
  SELECT ano, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
  SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
  FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm`
  GROUP BY ano ORDER BY ano
);


#pela via de transporte, agora com percentual
#percentual do usa das via de transporte pelo o valor e  volume importado
SELECT meio_transporte, sum(valor_dolar)*100/(2169120802018) AS percentual_valor_via, 
sum(peso_liquido)*100/(1684227798719) AS percentual_volume_via
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
GROUP BY meio_transporte ORDER BY percentual_volume_via DESC;

#resultado esquisito no volume postal, olhar a contagem 
SELECT SUM(peso_liquido) AS volume_total, COUNT(peso_liquido) AS num_casos,
sum(peso_liquido)/(1684227798719) AS percentagem
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE meio_transporte = 'POSTAL';
#valor pequeno, está em notação cinetifica

#porcetual do uso das vias em importação em 2022 
#valor total em 2022(250808584820) e volume total em (158139630507)
SELECT meio_transporte, 
sum(valor_dolar) AS valor_total, sum(valor_dolar)*100/(250808584820) AS percentual_valor_via, 
sum(peso_liquido) AS volume_total, sum(peso_liquido)*100/(158139630507) AS percentual_volume_via
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` 
WHERE ano =2022 GROUP BY meio_transporte ORDER BY percentual_volume_via DESC;


#de acordo com as GCCE - Grandes Categorias Ecomonicas
#valor total e volume total importado em cada categoria do nível 1 (5 categorias)
SELECT c.cgce_n1, 
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada categoria do nível 2 (13 categorias)- mais interresante para gráfico
SELECT c.cgce_n2, 
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada categoria do nível 2 no ano de 2022
SELECT c.cgce_n2, 
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(250808584820) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(158139630507) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm WHERE ano=2022
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada categoria do nível 3 (19 categorias)
SELECT c.cgce_economica_n3,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#de acordo com o ISIC - classificação internacional das industrias
#valor total e volume total importado em cada seção do ISIC (4 sessões) - usar as sessões como filtros
SELECT c.isic_secao,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#100 produtos mais importados da Indústria de Transformação (filtro)
SELECT i.produto, SUM(i.valor_dolar) AS valor_total, AVG(i.valor_dolar) AS media_valor, 
SUM(i.peso_liquido) AS volume_total, AVG(i.peso_liquido) AS media_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
WHERE c.isic_secao = 'Indústria de Transformação'
GROUP BY i.produto ORDER BY valor_total DESC LIMIT 100;

#os 50 produtos mais importados da Indústria de Transformação em 2022
SELECT i.produto, SUM(i.valor_dolar) AS valor_total, AVG(i.valor_dolar) AS media_valor, 
SUM(i.peso_liquido) AS volume_total, AVG(i.peso_liquido) AS media_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
WHERE c.isic_secao = 'Indústria de Transformação' AND i.ano=2022
GROUP BY i.produto ORDER BY valor_total DESC LIMIT 50;

#valor total e volume total importado em cada seção do ISIC (39 divisões)
SELECT c.isic_divisao,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada grupo do ISIC (94 grupos)
SELECT c.isic_grupo,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada classificação do ISIC (172 classificações)
SELECT c.isic_class_industrial,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#de acordo com o CUCI - 
#valor total e volume total importado em cada seção do CUCI (10 secões) 
SELECT c.cuci_secao,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada divisão do CUCI (66 divisão) - lista interresante
SELECT c.cuci_divisao,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada divisão do CUCI no ano de 2022 - só foram 65 divisões diferentes.
SELECT c.cuci_divisao,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(250808584820) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(158139630507) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm WHERE im.ano=2022
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada grupo do CUCI (302 grupos) - lista mais detalhada
SELECT c.cuci_grupo,
SUM(im.valor_dolar) AS valor_total, SUM(im.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(im.peso_liquido) AS volume_total, SUM(im.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#ver a classificação CUCI para os produtos na base combustivies
SELECT distinct(im.produto), c.cuci_item, c.cuci_subgrupo, c.cuci_grupo, c.cuci_divisao, c.cuci_secao
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
WHERE mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000)
ORDER BY c.cuci_subgrupo;


#de acordo com o SH - sistema harmonizado 
#valor total e volume total importado em cada secão do SH6 (21 secões) 
SELECT c.sh6_secao,
SUM(i.valor_dolar) AS valor_total, SUM(i.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(i.peso_liquido) AS volume_total, SUM(i.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada capitulos do SH6 ou (SH2) (96 capitulos) -lista compreensiva dos protudos
SELECT c.sh6_capitulo,
SUM(i.valor_dolar) AS valor_total, SUM(i.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(i.peso_liquido) AS volume_total, SUM(i.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC;

#valor total e volume total importado em cada posições do SH6 dentro da cada capitulo ou (SH4), limitado pelo 100 com maior valor
SELECT c.sh6_posicao, 
SUM(i.valor_dolar) AS valor_total, SUM(i.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(i.peso_liquido) AS volume_total, SUM(i.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
GROUP BY 1 ORDER BY valor_total DESC LIMIT 100;

#valor total e volume total importado em cada posições do SH6 dentro da cada capitulo ou (SH4), limitado pelo 100 com maior volume
SELECT c.sh6_posicao,
SUM(i.valor_dolar) AS valor_total, SUM(i.valor_dolar)*100/(2169120802018) AS porcento_valor, 
SUM(i.peso_liquido) AS volume_total, SUM(i.peso_liquido)*100/(1684227798719) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm 
GROUP BY 1 ORDER BY volume_total DESC LIMIT 100;
#15° lugar - milho, 8 - trigo

#valor total e volume total importado em cada posições do SH6 dentro da cada capitulo ou (SH4), 
#limitado pelo 100 com maior volume, no ano de 2022
SELECT c.sh6_posicao,
SUM(i.valor_dolar) AS valor_total, SUM(i.valor_dolar)*100/(250808584820) AS porcento_valor, 
SUM(i.peso_liquido) AS volume_total, SUM(i.peso_liquido)*100/(158139630507) AS porcento_volume
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON i.mercadoria=c.ncm  WHERE i.ano=2022
GROUP BY 1 ORDER BY volume_total DESC LIMIT 100;


##slq usado em combustívies

In [ ]:
# 10 MAIORES PREÇOS, SELECIONAR PRODUTO E ANO: gasolina comum no ano de 2022
SELECT data_final as Data_levantamento, estado as Estado, municipio as Municipio, preco_medio_revenda as Preco_medio
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
WHERE produto = 'GASOLINA COMUM' AND data_final >= '2022-01-01 00:00:00' and data_final < '2023-01-01 00:00:00'
ORDER BY preco_medio_revenda DESC LIMIT 10;

# 10 MENORES PREÇOS, SELECIONAR PRODUTO E ANO: gasolina comun no ano de 2022
SELECT data_final as Data_levantamento, estado as Estado, municipio as Municipio, preco_medio_revenda as Preco_medio
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
WHERE produto = 'GASOLINA COMUM' AND data_final >= '2022-01-01 00:00:00' and data_final < '2023-01-01 00:00:00'
ORDER BY preco_medio_revenda ASC LIMIT 10;

#Os estados mais pesquisados, mais representados
SELECT estado, COUNT(*) AS num_registros, SUM(num_postos_pesquisados) AS num_postos_pesquisados
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022
GROUP BY 1 ORDER BY num_registros DESC;

#Preço minimo, médio e máximo por estado, SELECIONAR PRODUTO E ANO: gasolina comun no ano de 2022
SELECT estado, MIN(preco_medio_revenda) AS menor_preco, AVG(preco_medio_revenda) AS preco_medio, MAX(preco_medio_revenda) AS maior_preco
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022
WHERE produto = 'GASOLINA COMUM' AND data_final >= '2022-01-01 00:00:00' and data_final < '2023-01-01 00:00:00'
GROUP BY 1 ORDER BY preco_medio DESC;

#Preço minimo e máximo por ano, SELECIONAR PRODUTO E ANO: gasolina comun
SELECT extract(year FROM data_final) AS ano, MIN(preco_medio_revenda) AS menor_preco, MAX(preco_medio_revenda) AS maior_preco
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022
WHERE produto = 'GASOLINA COMUM' GROUP BY ano ORDER BY ano DESC;

#Preço máximo por estado por ano, SELECIONAR PRODUTO: gasolina comun
SELECT extract(year FROM data_final) AS ano, estado, MAX(preco_medio_revenda) AS preco_maximo
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
WHERE produto = 'GASOLINA COMUM' GROUP BY ano, estado ORDER BY ano DESC, preco_maximo DESC;

#Preço mínimo, médio e máximo por ano e regiao dos protudos
SELECT extract(year FROM data_final) AS ano, produto, regiao, 
MIN(preco_medio_revenda) AS preco_minimo, AVG(preco_medio_revenda) AS preco_medio,MAX(preco_medio_revenda) AS preco_maximo
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
GROUP BY ano, produto, regiao ORDER BY ano DESC, preco_medio;

#Preço minimo e máximo por ano de todos os produtos
SELECT extract(year FROM data_final) AS ano, produto, MIN(preco_medio_revenda) AS menor_preco, MAX(preco_medio_revenda) AS maior_preco
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022
GROUP BY ano, produto ORDER BY ano DESC, menor_preco;

#Preço médio anual por produto
SELECT produto, extract(year FROM data_final) AS ano, AVG(preco_medio_revenda) AS preco_medio_anual
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022
GROUP BY produto, ano;

#Preço médio mensal por produto
SELECT produto, extract(year FROM data_final) AS ano, extract(month FROM data_final) AS mes, AVG(preco_medio_revenda) AS preco_medio_mensal
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
GROUP BY produto, ano, mes ORDER BY ano DESC;

#sobre quantidade vendida
#Volume vendido por ano dos produto
SELECT ano, produto, SUM(qtde_vendas) AS litros_vendidos 
FROM `bc26-ed7-projeto-final.comercio_brasil.volume_preco_combustiveis_mensal_2013_2022`
GROUP BY produto, ano ORDER BY ano DESC, litros_vendidos DESC;

#Volume vendido por mes dos produtos nos postos pesquisados
SELECT produto, ano, mes, SUM(qtde_vendas) AS litros_vendidos 
FROM `bc26-ed7-projeto-final.comercio_brasil.volume_preco_combustiveis_mensal_2013_2022`
GROUP BY ano, mes, produto ORDER BY ano DESC, produto, litros_vendidos DESC;

#Volume vendido por ano dos produto por estado, junto com o volume médio vendido por mes
SELECT ano, produto, estado, SUM(qtde_vendas) AS litros_vendidos, AVG(qtde_vendas) AS volume_medio_mensal 
FROM `bc26-ed7-projeto-final.comercio_brasil.volume_preco_combustiveis_mensal_2013_2022`
GROUP BY produto, ano, estado ORDER BY ano DESC, produto, litros_vendidos DESC;


##query enviar ao looker

In [ ]:
#Filtro com as tabelas de exportação e  importação e combustívies para o looker

#colunas da base de exportação por municpipio - 29422
SELECT ano, mes, SH4, cod_pais, estado, municipio, valor_dolar, peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_mun`
WHERE SH4=2710 or SH4=2711 or SH4=3826;

#colunas da base de importação por municipio - 42685
SELECT ano, mes, SH4,descricoes__sistema_harmonizado, cod_pais, estado, municipio, valor_dolar, peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_mun`
WHERE SH4=2710 or SH4=2711 or SH4=3826;


#colunas de classificação, resulta em 8 produtos
Select ncm, produto, sh6_mercadoria
FROM`bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes`
WHERE ncm =27100029 or ncm =27100041 or ncm =27100042 or ncm =27101259 or ncm =27111100 or ncm =27111910 or ncm =27112100 or ncm = 38260000;

##queries para exportar para o looker
#tabela de exportação com o filtro
CREATE OR REPLACE TABLE comercio_brasil.exp_prod_comb AS (SELECT e.ano, e.mes, c.produto, p.pais, e.estado, e.meio_transporte, e.valor_dolar, e.peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` AS e
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON e.cod_mercadoria = c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON e.cod_pais = p.cod_pais
WHERE c.ncm =27100029 or c.ncm =27100041 or c.ncm =27100042 or c.ncm =27101259 or c.ncm =27111100 or c.ncm =27111910 or c.ncm =27112100 or c.ncm = 38260000);

#tabela de exportação com o filtro
CREATE OR REPLACE TABLE comercio_brasil.imp_prod_comb AS (SELECT i.ano, i.mes, c.produto, p.pais, i.estado, i.meio_transporte, i.valor_dolar, i.peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON i.mercadoria = c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais = p.cod_pais
WHERE ano !=2012 AND
c.ncm =27100029 or c.ncm =27100041 or c.ncm =27100042 or c.ncm =27101259 or c.ncm =27111100 or c.ncm =27111910 or c.ncm =27112100 or c.ncm = 38260000);

#tabela de combustíveis
SELECT data_inicial, data_final, produto, preco_medio_revenda, regiao, estado, municipio, num_postos_pesquisados, unidade_medida FROM `bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022`;



#colunas escolhidas para ir para o looker
Select ncm, produto, sh6_mercadoria, sh6_capitulo, sh6_secao, cuci_divisao, cuci_secao, cgce_n2, cgce_n1, isic_secao, siit_setor_industrial
FROM`bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes`;

#usar apenas por produto para enviar para o looker
#colunas da base de exportação por produto - 12 084 296
SELECT ano, mes, cod_mercadoria, mercadoria, cod_pais, estado, meio_transporte, valor_dolar, peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm`;

#colunas da base de importação por produto - 18 560 388
SELECT ano, mes, mercadoria, produto, cod_pais, estado, meio_transporte, valor_dolar, peso_liquido
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` WHERE ano !=2012;


#combinar exportação por produto com classificação - 12 071 288
CREATE OR REPLACE TABLE comercio_brasil.class_imp_prod AS (SELECT e.ano, e.mes, mercadoria, p.pais, e.estado, e.meio_transporte, e.valor_dolar, e.peso_liquido,
c.sh6_capitulo, c.sh6_secao, c.cuci_divisao, c.cuci_secao, c.cgce_n2, c.isic_secao, c.siit_setor_industrial
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` AS e
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON e.cod_mercadoria = c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON e.cod_pais = p.cod_pais);

#combinar importação por produto com classificação - 18 526 543
CREATE OR REPLACE TABLE comercio_brasil.class_exp_prod AS (SELECT i.ano, i.mes, i.produto, p.pais, i.estado, i.meio_transporte, i.valor_dolar, i.peso_liquido,
c.sh6_capitulo, c.sh6_secao, c.cuci_divisao, c.cuci_secao, c.cgce_n2, c.isic_secao, c.siit_setor_industrial
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON i.mercadoria = c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais = p.cod_pais WHERE i.ano !=2012);


#contar as linhas
SELECT count(*)
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON i.mercadoria=c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais = p.cod_pais WHERE i.ano !=2012;

Select sh6_mercadoria, produto
FROM`bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` limit 50;




#query com exportação
#relação entre o meio de transporte o estado que exporta ordenado pelo valor total
SELECT meio_transporte, estado, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY meio_transporte ,valor_total DESC;

#relação entre o meio de transporte o estado que exporta ordenado pelo volume total
SELECT meio_transporte, estado, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY meio_transporte ,volume_total DESC;
#a ordem dos estados muda em alguns meios

#uso de cada meio de transporte por ano ordenado por valor
SELECT ano, meio_transporte, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY ano DESC,valor_total DESC;

#uso de cada meio de transporte por ano ordenado por volume
SELECT ano, meio_transporte, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY ano DESC,volume_total DESC;

#uso de cada meio de transporte  ordenado por valor
SELECT meio_transporte, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1 ORDER BY valor_total DESC;

#uso de cada meio de transporte ordenado por volume
SELECT meio_transporte, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1 ORDER BY volume_total DESC;




#acrescentando o valor por volume desses produtos nos 10 anos
SELECT mercadoria, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, (sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm`
WHERE cod_mercadoria in (27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000)
GROUP BY mercadoria ORDER BY dolar_por_volume DESC LIMIT 10;

##querys selecionadas para a documentação

In [ ]:
#combinar importação por produto com classificação, selecionar colunas para o looker
CREATE OR REPLACE TABLE looker.imp_prod AS (SELECT i.ano, i.mes, i.produto, p.pais, i.estado, i.meio_transporte, 
i.data, i.valor_dolar AS valor_importado, i.peso_liquido AS volume_importado,
c.sh6_capitulo, c.sh6_secao, c.cuci_divisao, c.cuci_secao, c.cgce_n2, c.isic_secao
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2013-2022_tratada_ncm` AS i
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c ON i.mercadoria = c.ncm
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON i.cod_pais = p.cod_pais);

#criar a tabela com o filtro para combustiveis para exportação, para combinar as análises e enviar ao looker 
CREATE OR REPLACE TABLE looker.exp_combustivel AS (SELECT e.ano, e.mes, e.mercadoria AS produto, p.pais, e.estado, e.meio_transporte, e.data, e.valor_dolar AS valor_exportado, e.peso_liquido AS volume_exportado
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` AS e
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.nome_pais` AS p ON e.cod_pais = p.cod_pais
WHERE e.cod_mercadoria = 27090010 or e.cod_mercadoria =27101922 or e.cod_mercadoria =99980101 or e.cod_mercadoria =99981002 or e.cod_mercadoria =27101259 or e.cod_mercadoria =27101921 or e.cod_mercadoria =27131100 or e.cod_mercadoria =27131200 or 
e.cod_mercadoria =27111100 or e.cod_mercadoria = 27101929 or e.cod_mercadoria =39111029 or e.cod_mercadoria =27111910);

#relação entre o meio de transporte o estado que exporta ordenado pelo volume total
SELECT meio_transporte, estado, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY meio_transporte ,volume_total DESC;

# 10 MENORES PREÇOS, SELECIONAR PRODUTO E ANO: gasolina comun no ano de 2022
SELECT data_final as Data_levantamento, estado as Estado, municipio as Municipio, preco_medio_revenda as Preco_medio
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
WHERE produto = 'GASOLINA COMUM' AND data_final >= '2022-01-01 00:00:00' and data_final < '2023-01-01 00:00:00'
ORDER BY preco_medio_revenda ASC LIMIT 10;

#Preço mínimo, médio e máximo por ano e regiao dos protudos
SELECT extract(year FROM data_final) AS ano, produto, regiao, 
MIN(preco_medio_revenda) AS preco_minimo, AVG(preco_medio_revenda) AS preco_medio,MAX(preco_medio_revenda) AS preco_maximo
FROM bc26-ed7-projeto-final.comercio_brasil.preco_combustiveis_2013_2022 
GROUP BY ano, produto, regiao ORDER BY ano DESC, preco_medio;

#os 50 maiores produtos valor por volume (dolar/volume) dos produtos nos 10 anos.
SELECT produto, sum(valor_dolar) as valor_total, sum(peso_liquido) as volume_total, 
(sum(valor_dolar)/sum(peso_liquido)) AS dolar_por_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2013-2022_tratada_ncm`
WHERE peso_liquido !=0 GROUP BY produto ORDER BY dolar_por_volume DESC LIMIT 50;

#insigh interresante, acho que fiz também por estado
#as via de transporte com de acordo com o volume exportado
SELECT meio_transporte, estado, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS valor_medio, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS volume_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm` 
GROUP BY 1, 2 ORDER BY meio_transporte, volume_total DESC;

#meses e anos ordenado pelo volume total importado - insight legal para ver em gráfico
SELECT ano, mes, sum(valor_dolar) as valor_total,  AVG(valor_dolar) AS valor_medio
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2013-2022_tratada_ncm` AS i
GROUP BY ano, mes ORDER BY valor_total DESC;

#valor total gasto em importação e o volume em relação a cada seção da Classificação Uniforme para Comercio Internacional em 2022
SELECT c.cuci_secao, SUM(im.valor_dolar) AS valor_total, SUM(im.peso_liquido) AS volume_total
FROM `bc26-ed7-projeto-final.comercio_brasil.imp_2012-2022_tratada_ncm` AS im
INNER JOIN `bc26-ed7-projeto-final.comercio_brasil.imp-exp-classificacoes` AS c
ON im.mercadoria=c.ncm 
WHERE ano=2022 GROUP BY 1 ORDER BY valor_total;

#meses com mais exportação em valor, junto com a media e o volume total e médio
SELECT mes, SUM(valor_dolar) AS valor_total, AVG(valor_dolar) AS media_valor, 
SUM(peso_liquido) AS volume_total, AVG(peso_liquido) AS media_volume 
FROM `bc26-ed7-projeto-final.comercio_brasil.exp_2013-2022_tratada_ncm`
GROUP BY mes ORDER BY valor_total;

